In [330]:
import pandas as pd
from tkinter import Tk, filedialog
import ast
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tabulate import tabulate
import pprint 
import json
import scipy.stats as stats

In [331]:
# Create Tk root
root = Tk()

# Hide the main window
root.withdraw()

# Set the main window to be always on top
root.call('wm', 'attributes', '.', '-topmost', True)

# Ask user to select CSV files
csv_paths = filedialog.askopenfilename(multiple=True, title='Select CSV files', filetypes=[('CSV files', '*.csv')])

# Initialize a list to store DataFrames for each selected CSV file
data_frames = []

# Iterate over each selected CSV file path
for csv_path in csv_paths:
    df = pd.read_csv(csv_path, sep=';')
    data_frames.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df.update(combined_df.applymap(lambda x: x.strip() if isinstance(x, str) else x))

# Now combined_df contains data from all selected CSV files
#print(combined_df)

In [332]:
# Define a list to store all selections and filtering criteria
filtering_criteria = []

print("Available headers:")
for idx, header in enumerate(combined_df.columns):
    print(f"{idx}: {header}")

# Prompt the user to choose a header by its number
header_number = int(input("Enter the number of the header you want to filter by: "))

# Check if the selected header is "array" and ask for input again if it is
while combined_df.columns[header_number] == "array":
    print("Sorry, you cannot select the 'array' column.")
    header_number = int(input("Enter the number of the header you want to filter by: "))

selected_header = combined_df.columns[header_number]

# Append the selected header to filtering_criteria
filtering_criteria.append(f"Selected Header: {selected_header}")

filter_value = input(f"Enter the value to filter by {selected_header}: ")

# Append the filter value to filtering_criteria
filtering_criteria.append(f"Filter Value: {filter_value}")

# Filtering rows based on the selected header and filter value
matching_rows = combined_df[combined_df[selected_header] == filter_value]

# Extracting the headers in the original order from the DataFrame
original_headers = matching_rows.columns.tolist()

print("Matching Rows:")
print(len(matching_rows))

# Display all selections and filtering criteria
print("\nAll Selections and Filtering Criteria:")
for criterion in filtering_criteria:
    print(criterion)



Available headers:
0: Mouse
1: Genotype
2: Date
3: Selected Behavior
4: Sham
5: Session Number
6: Fiber Connection
7: Right Turn Array
8: Right Turn Array Continue
9: Total Right Turn Bout Count
10: Total Right Turn Stim Count
11: Average Duration of Right Turn(seconds)
12: Total Right Turn Stim in Stim Block
13: Total Right Turn Bouts in Stim Block
14: Right Turn Average Duration in Stim Block(seconds)
15: Locomotion Array
16: Locomotion Array Continue
17: Total Locomotion Bout Count
18: Total Locomotion Stim Count
19: Average Duration of Locomotion(seconds)
20: Total Locomotion Stim in Stim Block
21: Total Locomotion Bouts in Stim Block
22: Locomotion Average Duration in Stim Block(seconds)
23: Face Groom Array
24: Face Groom Array Continue
25: Total Face Groom Bout Count
26: Total Face Groom Stim Count
27: Average Duration of Face Groom(seconds)
28: Total Face Groom Stim in Stim Block
29: Total Face Groom Bouts in Stim Block
30: Face Groom Average Duration in Stim Block(seconds)
Ent

In [334]:
 def get_possible_indices(tuples):
    index_to_label = {
        0: "Behavior,0",
        1: "Stim Count,1",
        2: "Frame Rate,2",
        3: "Bouts,3",
        4: "Duration,4", 
        5: "Start Time,5", 
        6: "End Time,6"

    }
    if tuples and isinstance(tuples[0], tuple):
        return [index_to_label[i] if i in index_to_label else f"Unknown Label {i}" for i in range(len(tuples[0]))]
    return []

def get_possible_indices_integers(tuples):
    if tuples and isinstance(tuples[0], tuple):
        return list(range(len(tuples[0])))
    return []

original_matching_rows = matching_rows.copy()

while True:
    start_filtering = input("Do you want to start filtering? (y/n): ")
    if start_filtering.lower() != 'y':
        print("Filtering process cancelled.")
        break
    
    print("\nAvailable headers:")
    for idx, header in enumerate(matching_rows.columns):
        print(f"{idx}: {header}")

    try:
        header_number = int(input("Enter the number of the header you want to filter by: "))
        
        if 0 <= header_number < len(matching_rows.columns):
            selected_header = matching_rows.columns[header_number]
            header_words = selected_header.split()
            filtering_criteria.append(f"Selected Header: {selected_header}")
            print(selected_header)
            
            substring = 'Array'
            if substring in selected_header:
                print(f"The substring '{substring}' was found in the header.")
            else:
                print(f"The substring '{substring}' was not found in the header.")

            if 'Array' in selected_header:
                tuple_cells = matching_rows[selected_header].tolist()
                #print(tuple_cells)
                int_1 = []

                for subli in tuple_cells:
                    intermediate = subli.strip('][').split('][')
                    int_1.append(intermediate)

                tuples_list = [[tuple(float(value) if '.' in value else int(value) for value in s.split(',')) for s in inner_list] for inner_list in int_1]
                #for x in tuples_list:
                    #print('Tuples List Before',len(x))
                # Find the index of selected_header in the column headers
                if selected_header in matching_rows.columns:
                    header_index = matching_rows.columns.get_loc(selected_header)

                    # Check if the next column index is valid
                    if header_index + 1 < len(matching_rows.columns):
                        next_column_name = matching_rows.columns[header_index + 1]
                        matching_columns_next = matching_rows[next_column_name]
                        tuple_cells_2 = matching_columns_next.tolist()
                        int_2 = []

                        for subli in tuple_cells_2:
                            if isinstance(subli, str):
                                intermediate = subli.strip('][').split('][')
                            elif isinstance(subli, int):
                                intermediate = [str(subli)]
                                
                            int_2.append(intermediate)

                    tuples_list_2 = [[tuple(float(value) if '.' in value else int(value) for value in s.split(',')) for s in inner_list] for inner_list in int_2]
            
                    values_added = False  # Initialize a flag to track if values were added

                    # Check the length of each tuple in tuplesst_2
                    for index, value in enumerate(tuples_list_2):
                        if len(value)== 1:
                            continue  # Skip single-value tuples

                        # Check if the index is within the bounds of tuples_list
                        if index < len(tuples_list):
                            tuples_list[index].extend(value)
                            values_added = True  # Set the flag to True if values were added
                                    # Check if values were added and inform the user
                    if values_added:
                        print("Values were added to tuples_list from tuples_list_2.")
                    else:
                        print("No values were added to tuples_list from tuples_list_2.")
                        
                    #for x in tuples_list:
                        #print('Tuples List After:', len(x))

                row_index_to_values = {}
                # Create a dictionary that associates row indices with values from tuples_list
                for row_index, row_data in enumerate(matching_rows.itertuples(), start=0):
                    row_identifier = row_data.Index  # Get the row identifier (usually the index)
                    row_index_to_values[row_identifier] = tuples_list[row_index]
                #for row_index, values in row_index_to_values.items():
                    #print(f"Row Index: {row_index}, Values: {values}")

                possible_indices = get_possible_indices(tuples_list[0])
                print("\nAvailable indices for the elements within tuples:")
                print(possible_indices)
                possible_indices_integers = get_possible_indices_integers(tuples_list[0])
                
                selected_indices_str = input("Enter the indices of the elements within each tuple to filter by (comma-separated): ")
                selected_indices = [int(idx.strip()) for idx in selected_indices_str.split(',')]
                selected_indices_with_range = []
                
                for selected_index in selected_indices:
                    if selected_index == 5 or selected_index == 6:
                        selected_indices_with_range.append(selected_index)
                    elif selected_index in possible_indices_integers:
                        selected_indices_with_range.append(selected_index)
                    else:
                        print(f"Invalid tuple element index {selected_index}. Skipping.")
                # Append selected indices to filtering_criteria
                filtering_criteria.append(f"Selected Indices: {selected_indices_with_range}") 
                
                if 5 in selected_indices_with_range or 6 in selected_indices_with_range:
                    range_filtering = input("Do you want to filter by an exact value or a range for index 5 or 6? (exact/range): ").lower()
                    if range_filtering == 'range':
                        range_min = int(input("Enter the minimum value of the range: "))
                        range_max = int(input("Enter the maximum value of the range: "))
                        # Append selected indices to filtering_criteria
                        filtering_criteria.append(f"Selected Indices: {selected_indices_with_range}")
                        matching_tuples = {}
                        for row_index, values in row_index_to_values.items():
                            for current_tuple in values:
                                value_at_selected_index = current_tuple[selected_index]
                                if range_min <= value_at_selected_index <= range_max:
                                    if row_index not in matching_tuples:
                                        matching_tuples[row_index] = []
                                        matching_tuples[row_index].append(current_tuple)
                                    else:
                                        matching_tuples[row_index].append(current_tuple)
                                        
                        if len(matching_tuples) > 0:
                            mask = []
                            for x in tuples_list:
                                found_true_for_tuple = False

                                for current_tuple in x:
                                    value_at_selected_index = current_tuple[selected_index]
                                    if range_min <= value_at_selected_index <= range_max:
                                        mask.append(True)
                                        found_true_for_tuple = True
                                        break

                                if not found_true_for_tuple:
                                    mask.append(False)

                            indices_to_keep = [i for i, value in enumerate(mask) if value]
                            filtered_matching_rows = matching_rows[matching_rows.index.isin(indices_to_keep)]
                            matching_rows = filtered_matching_rows
                            original_matching_rows = matching_rows.copy()
                        else:
                            print("There are no matching tuples and no matching rows")
                            if len(matching_rows) == 0:
                                matching_rows = original_matching_rows.copy()
                                print('Matching Rows Data Frame Updated to Last Successful Filter')

                    else:
                        user_input = input("Enter the value to filter by in the selected element at index 5 or 6 (or 'all' to select all): ")
                        if user_input.lower() == 'all':
                            matching_tuples = {}
                            for row_index, values in row_index_to_values.items():
                                for current_tuple in values:
                                    if row_index not in matching_tuples:
                                        matching_tuples[row_index] = []
                                        matching_tuples[row_index].append(current_tuple)
                                    else:
                                        matching_tuples[row_index].append(current_tuple)
                                                
                        else:
                            try:
                                user_input = int(user_input)
                                matching_tuples = {}
                                for row_index, values in row_index_to_values.items():
                                    for current_tuple in values:
                                        value_at_selected_index = current_tuple[selected_index]
                                        if value_at_selected_index == user_input:
                                            if row_index not in matching_tuples:
                                                matching_tuples[row_index] = []
                                                matching_tuples[row_index].append(current_tuple)
                                            else:
                                                matching_tuples[row_index].append(current_tuple)
                                                
                                if len(matching_tuples) > 0:
                                    mask = []
                                    for x in tuples_list:
                                        found_true_for_tuple = False

                                        for current_tuple in x:
                                            value_at_selected_index = current_tuple[selected_index]
                                            if value_at_selected_index == user_input:
                                                mask.append(True)
                                                found_true_for_tuple = True
                                                break

                                        if not found_true_for_tuple:
                                            mask.append(False)

                                    indices_to_keep = [i for i, value in enumerate(mask) if value]
                                    filtered_matching_rows = matching_rows[matching_rows.index.isin(indices_to_keep)]
                                    matching_rows = filtered_matching_rows
                                    original_matching_rows = matching_rows.copy()
                                    print('Matching Rows Data Frame and Original Copy Data Frame Are Updated')
                                else:
                                    print("There are no matching tuples and no matching rows")
                                    if len(matching_rows) == 0:
                                        matching_rows = original_matching_rows.copy()
                                        print('Matching Rows Data Frame Replaced with Last Successful Filter')

                            except ValueError:
                                print(f"Invalid input for index 5 or 6. Please enter a valid value of the expected data type.")

                else:
                    for selected_index in selected_indices_with_range:
                        if selected_index != 5 or selected_index != 6:
                            example_tuple = tuples_list[0]
                            for x in example_tuple[:7]:
                                print("Examples Tuples:", x)

                            values = set()
                            for x in tuples_list:
                                for y in x:
                                    if len(y) > selected_index:
                                        values.add(y[selected_index])
                            print(f"\nPreview of possible values for the selected element at index {selected_index}:")
                            print(values)
                            
                            user_input = input(f"Enter the value to filter by in the selected element at index {selected_index} (or 'all' to select all): ")

                            if user_input.lower() == 'all':
                                matching_tuples = {}
                                for row_index, values in row_index_to_values.items():
                                    for current_tuple in values:
                                        if row_index not in matching_tuples:
                                            matching_tuples[row_index] = []
                                            matching_tuples[row_index].append(current_tuple)
                                        else:
                                            matching_tuples[row_index].append(current_tuple)
                            else:
                                try:
                                    user_input = int(user_input)
                                    matching_tuples = {}
                                    for row_index, values in row_index_to_values.items():
                                        for current_tuple in values:
                                            value_at_selected_index = current_tuple[selected_index]
                                            if value_at_selected_index == user_input:
                                                if row_index not in matching_tuples:
                                                    matching_tuples[row_index] = []
                                                    matching_tuples[row_index].append(current_tuple)
                                                else:
                                                    matching_tuples[row_index].append(current_tuple)
                                                
                                    if len(matching_tuples) > 0:
                                        mask = []
                                        for x in tuples_list:
                                            found_true_for_tuple = False

                                            for current_tuple in x:
                                                value_at_selected_index = current_tuple[selected_index]
                                                if value_at_selected_index == user_input:
                                                    mask.append(True)
                                                    found_true_for_tuple = True
                                                    break

                                            if not found_true_for_tuple:
                                                mask.append(False)

                                        indices_to_keep = [i for i, value in enumerate(mask) if value]
                                        filtered_matching_rows = matching_rows[matching_rows.index.isin(indices_to_keep)]
                                        matching_rows = filtered_matching_rows
                                        original_matching_rows = matching_rows.copy()
                                    else:
                                        print("There are no matching tuples and no matching rows")
                                        if len(matching_rows) == 0:
                                            matching_rows = original_matching_rows.copy()
                                            print('Matching Rows Data Frame Updated to Last Successful Filter')

                                except ValueError:
                                    print(f"Invalid input for index {selected_index}. Please enter a valid value of the expected data type.")
            else:
                possible_values = matching_rows[selected_header].unique()
                print(f"Unique possible values for {selected_header}:")
                for value in possible_values:
                    print(value)

                filter_value = input(f"Enter the value to filter by {selected_header}: ")
                filtering_criteria.append(f"Filter Value: {filter_value}")
                
                selected_value_type = type(matching_rows[selected_header].iloc[0])
                try:
                    converted_user_input = selected_value_type(filter_value)
                    matching_rows = matching_rows[matching_rows[selected_header] == converted_user_input]

                    if len(matching_rows) == 0:
                        print("No matches found.")
                        matching_rows = original_matching_rows.copy()
                    else:
                        original_matching_rows = matching_rows.copy()
                except ValueError:
                    print("Invalid input. Please enter a valid value of the expected data type.")

            continue_input = input("Do you want to continue? (y/n): ")
            if continue_input.lower() == 'n':
                break
        else:
            print("Invalid header number. Please enter a valid number.")
    except ValueError as e:
        print("An error occurred:", e)
        
filtering_criteria.append(f"Matching Rows Count: {len(matching_rows)}")
# Create a set of row indices in matching_rows
matching_rows_indices = set(matching_rows.index)

# Create a copy of matching_tuples to iterate over while removing rows
matching_tuples_copy = matching_tuples.copy()

# Iterate over the keys (row indices) in matching_tuples
for row_index in matching_tuples_copy.keys():
    # Check if the row_index is not in matching_rows_indices
    if row_index not in matching_rows_indices:
        # Remove the row from matching_tuples
        del matching_tuples[row_index]

# Now, matching_tuples only contains rows that are both in matching_tuples and matching_rows                 
# Display all selections and filtering criteria
print("\nAll Selections and Filtering Criteria:")
for criterion in filtering_criteria:
    print(criterion)


Do you want to start filtering? (y/n): y

Available headers:
0: Mouse
1: Genotype
2: Date
3: Selected Behavior
4: Sham
5: Session Number
6: Fiber Connection
7: Right Turn Array
8: Right Turn Array Continue
9: Total Right Turn Bout Count
10: Total Right Turn Stim Count
11: Average Duration of Right Turn(seconds)
12: Total Right Turn Stim in Stim Block
13: Total Right Turn Bouts in Stim Block
14: Right Turn Average Duration in Stim Block(seconds)
15: Locomotion Array
16: Locomotion Array Continue
17: Total Locomotion Bout Count
18: Total Locomotion Stim Count
19: Average Duration of Locomotion(seconds)
20: Total Locomotion Stim in Stim Block
21: Total Locomotion Bouts in Stim Block
22: Locomotion Average Duration in Stim Block(seconds)
23: Face Groom Array
24: Face Groom Array Continue
25: Total Face Groom Bout Count
26: Total Face Groom Stim Count
27: Average Duration of Face Groom(seconds)
28: Total Face Groom Stim in Stim Block
29: Total Face Groom Bouts in Stim Block
30: Face Groom A

In [335]:
print(len(matching_rows))
for row_index, values in matching_tuples.items():
    print('Number of matching tuples:', len(values))

6


In [336]:
print(matching_rows)


   Mouse Genotype   Date Selected Behavior Sham  Session Number  \
3   AD10       D1  62623        Locomotion    n               1   
4   AD10       D1  72723        Locomotion    n               2   
16  AD11       D1  51223        Locomotion    n               1   
17  AD11       D1  72523        Locomotion    n               2   
51   P22       D1  62923        Locomotion    n               1   
52   P22       D1  72523        Locomotion    n               2   

   Fiber Connection                                   Right Turn Array  \
3         Bilateral  [1,0,3,1,1,35,36][1,1,9,2,3,59,62][1,1,6,3,2,6...   
4         Bilateral  [1,1,9,1,3,21,24][1,1,9,2,3,30,33][1,1,6,3,2,4...   
16        Bilateral  [1,2,18,1,6,121,127][1,1,12,2,4,158,162][1,1,6...   
17        Bilateral  [1,0,3,1,1,77,78][1,2,18,2,6,83,89][1,0,3,3,1,...   
51        Bilateral  [1,3,27,1,9,23,32][1,1,9,2,3,39,42][1,0,3,3,1,...   
52        Bilateral  [1,1,6,1,2,9,11][1,0,3,2,1,21,22][1,1,9,3,3,25...   

   Right Tu

In [322]:
# Define a function to determine two different keys based on conditions
def determine_key(selected_header, unique_header_3_value, unique_header_4_value,unique_header_0_value,unique_header_10_value, unique_header_1_value,unique_header_6_value):
    # Split the selected_header by spaces and enumerate the strings
    header_parts = selected_header.split()
    print(header_parts)
    for i, part in enumerate(header_parts):
        if part == "Array":
            if i >= 2:
                new_selected_header = f"{header_parts[i - 2]} {header_parts[i - 1]}"
            else:
                new_selected_header = f"{header_parts[i - 1]}"

        # Initialize two keys with common components
    key_1 = f"{unique_header_1_value} {unique_header_3_value} {new_selected_header} Bout Count {unique_header_0_value}"
    key_2 = f"{unique_header_1_value} {unique_header_3_value} {new_selected_header} Duration {unique_header_0_value}"
    # Check if unique_header_4_value is equal to "y" for both keys
    if unique_header_4_value == "y":
        key_1 = f"{unique_header_1_value} {new_selected_header} Bout Count {unique_header_10_value}"
        key_2= f"{unique_header_1_value} {new_selected_header} Duration {unique_header_10_value}"
    else:
        # Check if "Right Turn" is in unique_header_3_value for key_1
        if "Right Turn" in unique_header_3_value:
            key_1 = f"{unique_header_1_value} {unique_header_3_value} {unique_header_6_value} {new_selected_header} Bout Count {unique_header_0_value }"
            key_2=f"{unique_header_1_value} {unique_header_3_value} {unique_header_6_value} {new_selected_header} Duration {unique_header_0_value }"

    return key_1,key_2

# Initialize the averages_dict
averages_dict = {}

# Load the existing averages_dict from the file if it exists
averages_dict_file = "Practice"
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    averages_dict = {}

# Get a list of unique headers
unique_headers = matching_rows.columns.tolist()  # Get all available columns as headers

# Uncomment one of the following lines based on your choice:

# Option 1: Specify header indices manually
header_indices_to_process = [7, 15, 23]

# Option 2: Choose a single header using user input
#print("\nAvailable headers:")
#for idx, header in enumerate(matching_rows.columns):
    #print(f"{idx}: {header}")
#header_selection = int(input("Enter the number of the selected header: "))
#header_indices_to_process = [header_selection] if 'header_selection' in locals() else []

# Initialize key1 and key2 with empty strings
key1 = ""

# Iterate over the selected header indices
for header_selection in header_indices_to_process:
    selected_header = matching_rows.columns[header_selection]

    # Filter the DataFrame to select all rows with the selected header
    filtered_rows = matching_rows[selected_header]

    # Set the values for unique_header_1_value, unique_header_3_value, and unique_header_4_value
    unique_header_1_value = matching_rows.iloc[0, 1]
    unique_header_3_value = matching_rows.iloc[0, 3]
    unique_header_4_value = matching_rows.iloc[0, 4]
    unique_header_6_value = matching_rows.iloc[0, 6]
    unique_header_0_value = "Stim"
    unique_header_10_value = "Sham"

    # Initialize lists to store average stim and post durations for each row
    average_boutcount = []
    average_duration = []
    # Process the filtered data for both "stim" and "post"
    for row_index, row in filtered_rows.items():
        # Initialize lists for durations in the current row for both "bout count"
        boutcount = []
        duration=[]

        # Remove the additional square brackets at the beginning and end of the row
        row = row.strip('[]')

        # Split the row into individual tuples using '][' as the delimiter
        tuples_list = row.split('][')

        # Iterate over the tuples in the list
        for tuple_str in tuples_list:
            # Remove the square brackets and split the tuple values by comma
            values = [float(val) for val in tuple_str.strip('[]').split(',')]

            # Now, 'values' contains the individual values as floats
            # You can access and manipulate the values based on your conditions
            if len(values) >= 7:
                value5 = values[5]
                value6 = values[6]

                # Apply conditions for "stim" analysis
                if value5 > 3000 and value6 <= 21000:
                    boutcount.append(values[3])  # Append the bout value to the stim_boutcount list
                    duration.append(values[4])
           # Print bout count and duration for the current row
       # print(f"Row {row_index}: Bout Count = {boutcount}, Duration = {duration}")
        # Initialize the variable to store the integer average
    
        average_boutcount_value = len(boutcount) if boutcount else None
        
        average_duration_value=(sum(duration)/len(duration)) if duration else None
        
        # Print the average bout count along with the row index (or any relevant identifier)
         # Print bout count and duration for the current row
        #print(f"Row {row_index}: Bout Count = {average_boutcount_value}, Duration = {average_duration_value}")
        # Append the average bout count value for this row to the list
        average_boutcount.append(average_boutcount_value)
        average_duration.append(average_duration_value)
        print(average_boutcount)
        print(average_duration)
        
        # Print the calculated averages for the current row
        #print(f"Row {row_index}: Average Stim Duration: {average_stim_duration}, Average Post Duration: {average_post_duration}")

    # Determine the key for the current row
    key1 = determine_key(selected_header, unique_header_3_value, unique_header_4_value, unique_header_0_value, unique_header_10_value, unique_header_1_value, unique_header_6_value)
    key2 = determine_key(selected_header, unique_header_3_value, unique_header_4_value, unique_header_0_value, unique_header_10_value, unique_header_1_value, unique_header_6_value)
    print(key1,key2)


    # Create a dictionary to hold the durations for the current row
    boutcount_data = {
        'bout count':average_boutcount,
        'duration count':average_duration
    }
    # Print the duration_data dictionary
    #print("Duration Data:")
    #print(duration_data)

    # Determine the keys for the current header
    key1, key2 = determine_key(selected_header, unique_header_3_value, unique_header_4_value, unique_header_0_value, unique_header_10_value, unique_header_1_value, unique_header_6_value)

    # Use key1 for bout counts and key2 for durations
    if key1 not in averages_dict:
        averages_dict[key1] = {}
    averages_dict[key1] = average_boutcount

    if key2 not in averages_dict:
        averages_dict[key2] = {}
    averages_dict[key2]= average_duration

# Save the averages_dict to a file
try:
    with open(averages_dict_file, 'w') as file:
        json.dump(averages_dict, file, separators=(',', ':'))
except Exception as e:
    print(f"An error occurred while saving the file: {e}")

[67]
[2.1044776119402986]
[67, 346]
[2.1044776119402986, 1.8988439306358382]
[67, 346, 492]
[2.1044776119402986, 1.8988439306358382, 1.7865853658536586]
['Right', 'Turn', 'Array']
['Right', 'Turn', 'Array']
('D1 Right Turn Ipsilateral Right Turn Bout Count Stim', 'D1 Right Turn Ipsilateral Right Turn Duration Stim') ('D1 Right Turn Ipsilateral Right Turn Bout Count Stim', 'D1 Right Turn Ipsilateral Right Turn Duration Stim')
['Right', 'Turn', 'Array']
[134]
[5.097014925373134]
[134, 359]
[5.097014925373134, 3.604456824512535]
[134, 359, 437]
[5.097014925373134, 3.604456824512535, 2.6338672768878717]
['Locomotion', 'Array']
['Locomotion', 'Array']
('D1 Right Turn Ipsilateral Locomotion Bout Count Stim', 'D1 Right Turn Ipsilateral Locomotion Duration Stim') ('D1 Right Turn Ipsilateral Locomotion Bout Count Stim', 'D1 Right Turn Ipsilateral Locomotion Duration Stim')
['Locomotion', 'Array']
[540]
[2.738888888888889]
[540, 311]
[2.738888888888889, 2.7781350482315115]
[540, 311, 242]
[2.738

In [325]:
# Define the path to the JSON file where you want to store the dictionary
p_value_file = "Bout x duration"
averages_dict_file = "Practice"


# Initialize the p_value_dict as a global variable or load it from the JSON file if it exists
try:
    with open(p_value_file, 'r') as file:
        p_value_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    p_value_dict = {}

# Initialize the averages_dict as a global variable or load it from the JSON file if it exists
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # Handle the file not found error appropriately
    averages_dict = {}

# Extract keys without 'Sham' and keys with 'Sham' from averages_dict
keys_bout = [key for key in averages_dict.keys() if 'Duration' not in key]
keys_duration = [key for key in averages_dict.keys() if 'Duration' in key]

# Check if there are keys without 'Sham' to use as Key1
if keys_bout:
    print("Available keys without 'Sham' (showing first 3 elements):")
    for idx, key in enumerate(keys_bout, start=1):
        # Split the key using space as the separator
        key_elements = key.split(' ')
        formatted_key = ' '.join(key_elements)

        print(f"{idx}. {formatted_key}")

    # Prompt the user to select Key1
    try:
        selected_key_idx1 = int(input("Enter the number corresponding to Key1: ")) - 1
        key1 = keys_bout[selected_key_idx1]
        print(f"You have selected Key1: {key1}")
        words = key1.split(' ')

        # Find the index of the word 'bout' in the list of words
        try:
            index_of_bout = words.index('Bout')
        except ValueError:
            index_of_bout = len(words)

        # Create a new list containing words up until 'bout'
        matching_criteria_1 = words[:index_of_bout]

        # Join the words back into a string if needed
        matching_criteria_1_str = ' '.join(matching_criteria_1)

        print(matching_criteria_1_str)

        auto_fill_keys = [key for key in keys_duration if matching_criteria_1 == key.split(' ')[:index_of_bout]]

        if auto_fill_keys:
            print("Auto-filled options for Key2:")
            for idx, key in enumerate(auto_fill_keys[:5], start=1):
                # Extract the first elements of the key based on index_of_bout
                key_elements = key.split(' ')[:index_of_bout]
                formatted_key = ' '.join(key_elements)
                print(f"{idx}. {formatted_key}")

            # Automatically set Key2 based on the matching criteria
            key2 = auto_fill_keys[0]  # You can adjust this if you have specific logic to choose among them
            print(f"Auto-filled Key2 based on matching criteria: {key2}")
            
            # Retrieve the samples associated with the selected keys
            bout_samples = averages_dict.get(key1, [])
            duration_samples = averages_dict.get(key2, [])
            print(f"Bouts: {bout_samples}")
            print(f"Duration:{duration_samples}")
            
            converted_duration_samples=[]
            
            for x in duration_samples:
                new_duration= x/10
                converted_duration_samples.append(new_duration)
           
            print(f"Converted Durations: {converted_duration_samples}")
            
            time_during_stim = []

            for index, (x, y) in enumerate(zip(bout_samples, converted_duration_samples)):
                minutes = (x * y)
                time_during_stim.append(minutes)

            print(time_during_stim)
            
            # Convert the tuple keys to strings before updating the p_value_dict
            str_key1 = ' '.join(matching_criteria_1)

            # Update the p_value_dict with the p-values using the selected keys
    
            p_value_dict[str_key1] = {
                'time in seconds':time_during_stim,
            }

            # Save the updated p_value_dict back to the JSON file
            with open(p_value_file, 'w') as file:
                json.dump(p_value_dict, file)
        else:
            print("No auto-filled options available based on Key1.")
    except ValueError:
        print("Invalid input for Key1. Please enter a valid number.")

Available keys without 'Sham' (showing first 3 elements):
1. D1 Locomotion Right Turn Bout Count Stim
2. D1 Locomotion Locomotion Bout Count Stim
3. D1 Locomotion Face Groom Bout Count Stim
4. D1 Face Groom Right Turn Bout Count Stim
5. D1 Face Groom Locomotion Bout Count Stim
6. D1 Face Groom Face Groom Bout Count Stim
7. D1 Right Turn Bilateral Right Turn Bout Count Stim
8. D1 Right Turn Bilateral Locomotion Bout Count Stim
9. D1 Right Turn Bilateral Face Groom Bout Count Stim
10. D1 Right Turn Contralateral Right Turn Bout Count Stim
11. D1 Right Turn Contralateral Locomotion Bout Count Stim
12. D1 Right Turn Contralateral Face Groom Bout Count Stim
13. D1 Right Turn Ipsilateral Right Turn Bout Count Stim
14. D1 Right Turn Ipsilateral Locomotion Bout Count Stim
15. D1 Right Turn Ipsilateral Face Groom Bout Count Stim
Enter the number corresponding to Key1: 15
You have selected Key1: D1 Right Turn Ipsilateral Face Groom Bout Count Stim
D1 Right Turn Ipsilateral Face Groom
Auto-filled

In [327]:
import json
import re
from tabulate import tabulate

# Read data from the JSON file
with open('Bout x duration', 'r') as json_file:
    data = json.load(json_file)

# Create lists to hold row and column labels
row_labels = sorted(set(re.sub(r'[^A-Za-z ]+', '', key).strip().lower() for key in data.keys()))
column_labels = sorted(set(event.lower() for event in data.keys()))

# Create a table to display the time in seconds
table_data = []

# Iterate through the data and populate the table
for row_label in row_labels:
    row = [row_label]
    for column_label in column_labels:
        key = f"{row_label} {column_label}"
        time_in_seconds = data.get(key, {}).get("time in seconds", [])
        time_str = ', '.join([f'{time:.2f}' for time in time_in_seconds])
        row.append(time_str)
    table_data.append(row)

# Print the table
table_headers = [""] + column_labels
table = tabulate(table_data, headers=table_headers, tablefmt="fancy_grid", numalign="center", stralign="center")

print(table)


╒═══════════════════════════════════════╤════════════════════════════╤════════════════════════════╤════════════════════════════╤════════════════════════════╤════════════════════════════╤════════════════════════════╤══════════════════════════════════════╤══════════════════════════════════════╤══════════════════════════════════════╤══════════════════════════════════════════╤══════════════════════════════════════════╤══════════════════════════════════════════╤════════════════════════════════════════╤════════════════════════════════════════╤════════════════════════════════════════╕
│                                       │  d1 face groom face groom  │  d1 face groom locomotion  │  d1 face groom right turn  │  d1 locomotion face groom  │  d1 locomotion locomotion  │  d1 locomotion right turn  │  d1 right turn bilateral face groom  │  d1 right turn bilateral locomotion  │  d1 right turn bilateral right turn  │  d1 right turn contralateral face groom  │  d1 right turn contralateral locomotion

## Using Columns In DataFrame

In [286]:
# Define a function to determine the key based on conditions
def determine_key(selected_header, unique_header_3_value, unique_header_4_value):
    # Split the selected_header by spaces and enumerate the strings
    header_parts = selected_header.split()
    for i, part in enumerate(header_parts):
        if part == "Stim":
            block="Stim Bock"
        if part == "Bouts":
            # Check the string before "Bouts"
            if i > 0 and header_parts[i - 1] == "Turn":
                new_selected_header = 'Right Turn Bouts'
            elif i > 0 and header_parts[i - 1] == "Groom":
                new_selected_header = 'Face Groom Bouts'
            else:
                new_selected_header = f"{header_parts[i - 1]} Bouts"

    # Check if unique header 3 contains "Right Turn"
    if "Right Turn" in unique_header_3_value:
        return [unique_header_1_value, unique_header_3_value, unique_header_6_value, new_selected_header,block]

    # Check if unique header 4 has value "y"
    if unique_header_4_value == "y":
        return [unique_header_1_value, new_selected_header, matching_rows.columns[4]]

    # Default key order
    return [unique_header_1_value, unique_header_3_value, new_selected_header,block]

# Initialize the averages_dict
averages_dict = {}

# Load the existing averages_dict from the file if it exists
averages_dict_file = "Stim_Sham_Inner_Session_dict_file"
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    averages_dict = {}

# Get a list of unique headers
unique_headers = matching_rows.columns.tolist()  # Get all available columns as headers

# Uncomment one of the following lines based on your choice:

# Option 1: Specify header indices manually
#header_indices_to_process = [13, 21, 29]

# Option 2: Choose a single header using user input
print("\nAvailable headers:")
for idx, header in enumerate(matching_rows.columns):
    print(f"{idx}: {header}")
    header_selection = int(input("Enter the number of the selected header: "))
header_indices_to_process = [header_selection] if 'header_selection' in locals() else []

# Iterate over the selected header indices
for header_selection in header_indices_to_process:
    selected_header = matching_rows.columns[header_selection]

    # Filter the DataFrame to select all rows with the selected header
    filtered_rows = matching_rows[selected_header]

    # Set the values for unique_header_1_value, unique_header_3_value, and unique_header_4_value
    unique_header_1_value = matching_rows.iloc[0, 1]
    unique_header_3_value = matching_rows.iloc[0, 3]
    unique_header_4_value = matching_rows.iloc[0, 4]
    unique_header_6_value = matching_rows.iloc[0, 6]

    # Use the determine_key function to determine the key based on conditions
    key_order = determine_key(selected_header, unique_header_3_value, unique_header_4_value)

    # Initialize a list for bout_count
    bout_count = []

    # Process the filtered data
    for value in filtered_rows:
        # Append value to the bout_count list
        bout_count.append(value)

    # Determine the key based on the selected_header and other conditions
    key = " ".join(key_order)
    print(key)
    # Append the bout_count to the list associated with the key
    #if key in averages_dict:
        #averages_dict[key].extend(bout_count)
    #else:
        #averages_dict[key] = bout_count

# Save the updated averages_dict to the file without double square brackets
#with open(averages_dict_file, 'w') as file:
    #json.dump(averages_dict, file, separators=(',', ':'))


Available headers:
0: Mouse


KeyboardInterrupt: Interrupted by user

In [99]:
# Define the path to the JSON file where you want to store the dictionary
p_value_file = "p_value_inner_trial_bout.json"
averages_dict_file = "Stim_Sham_Inner_Session_dict_file"

# Initialize the p_value_dict as a global variable or load it from the JSON file if it exists
try:
    with open(p_value_file, 'r') as file:
        p_value_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    p_value_dict = {}

# Initialize the averages_dict as a global variable or load it from the JSON file if it exists
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # Handle the file not found error appropriately
    averages_dict = {}

# Extract keys without 'Sham' and keys with 'Sham' from averages_dict
keys_without_sham = [key for key in averages_dict.keys() if 'Sham' not in key]
keys_with_sham = [key for key in averages_dict.keys() if 'Sham' in key]

# Check if there are keys without 'Sham' to use as Key1
if keys_without_sham:
    print("Available keys without 'Sham' (showing first 3 elements):")
    for idx, key in enumerate(keys_without_sham, start=1):
        # Split the key using space as the separator
        key_elements = key.split(' ')

        # Check if element 2 is "Right" and combine it with elements 3 and 4
        if len(key_elements) >= 4 and key_elements[1] == 'Right':
            formatted_key = ' '.join([key_elements[0], key_elements[1] + ' ' + key_elements[2] + ' ' + key_elements[3], key_elements[4]])
        elif len(key_elements) >= 3 and key_elements[1] == 'Face':
            formatted_key = ' '.join([key_elements[0], key_elements[1] + ' ' + key_elements[2], key_elements[3]])
        else:
            formatted_key = ' '.join(key_elements[:3])

        print(f"{idx}. {formatted_key}")

    # Prompt the user to select Key1
    try:
        selected_key_idx1 = int(input("Enter the number corresponding to Key1: ")) - 1
        key1 = keys_without_sham[selected_key_idx1]
        print(f"You have selected Key1: {key1}")

        # Auto-fill Key2 based on the matching criteria
        matching_criteria_1 = key1.split(' ')
        print(matching_criteria_1)
        if matching_criteria_1[1] == 'Right':
            matching_criteria = [matching_criteria_1[0], matching_criteria_1[4]]
            auto_fill_keys = [key for key in keys_with_sham if matching_criteria == [key.split(' ')[0], key.split(' ')[1]]]
        elif matching_criteria_1[1] == 'Face':
            matching_criteria = [matching_criteria_1[0], matching_criteria_1[3]]
            auto_fill_keys = [key for key in keys_with_sham if matching_criteria == [key.split(' ')[0], key.split(' ')[1]]]
        else:
            matching_criteria = [matching_criteria_1[0], matching_criteria_1[2]]
            auto_fill_keys = [key for key in keys_with_sham if matching_criteria == [key.split(' ')[0], key.split(' ')[1]]]

        if auto_fill_keys:
            print("Auto-filled options for Key2:")
            for idx, key in enumerate(auto_fill_keys[:5], start=1):
                # Extract the first three elements of the key
                key_elements = key.split(' ')[:4]
                formatted_key = ' '.join(key_elements)
                print(f"{idx}. {formatted_key}")

            # Automatically set Key2 based on the matching criteria
            key2 = auto_fill_keys[0]  # You can adjust this if you have specific logic to choose among them
            print(f"Auto-filled Key2 based on matching criteria: {key2}")

            # Retrieve the samples associated with the selected keys
            samples1 = averages_dict.get(key1, [])
            samples2 = averages_dict.get(key2, [])

            # Perform bootstrapping with 10,000 iterations
            num_bootstrap_samples = 10000
            bootstrap_mean_differences = []

            # Perform bootstrapping with 'num_bootstrap_samples' iterations
            for _ in range(num_bootstrap_samples):
                # Generate bootstrap samples by resampling with replacement
                bootstrap_samples1 = np.random.choice(samples1, len(samples1), replace=True)
                bootstrap_samples2 = np.random.choice(samples2, len(samples2), replace=True)

                # Calculate the mean difference between bootstrap_samples1 and bootstrap_samples2
                bootstrap_mean_difference = np.mean(bootstrap_samples1) - np.mean(bootstrap_samples2)

                # Append the mean difference to the list
                bootstrap_mean_differences.append(bootstrap_mean_difference)

            # Calculate the observed difference in means between the original samples
            observed_diff = np.mean(samples1) - np.mean(samples2)

            # Calculate the p-value for the bootstrap test
            bootstrap_p_value = (np.sum(np.abs(bootstrap_mean_differences) >= np.abs(observed_diff)) + 1) / (num_bootstrap_samples + 1)

            # Perform a t-test on the original samples
            t_statistic_ttest, p_value_ttest = stats.ttest_ind(samples1, samples2)

            # Perform Welch's t-test
            t_statistic_welch, p_value_welch = stats.ttest_ind(samples1, samples2, equal_var=False)

            # Print the t-test results
            print("T-Test Results:")
            print("t-statistic (T-Test):", t_statistic_ttest)
            print("p-value (T-Test):", p_value_ttest)

            # Print Welch's T-Test Results
            print("Welch's T-Test Results:")
            print("t-statistic (Welch's T-Test):", t_statistic_welch)
            print("p-value (Welch's T-Test):", p_value_welch)

            # Print the bootstrapping results
            print("Bootstrapping Results:")
            print("Bootstrap p-value (Bootstrapping):", bootstrap_p_value)

            # Determine whether the t-test results are statistically significant (typically p < 0.05)
            if round(p_value_ttest, 2) <= 0.05:
                print(f"The means of {key1} and {key2} are statistically different (T-Test).")
            else:
                print(f"There is no significant difference between the means of {key1} and {key2} (T-Test).")

            # Determine whether the Mann-Whitney U test results are statistically significant (typically p < 0.05)
            if round(bootstrap_p_value, 2) <= 0.05:
                print(f"The distributions of {key1} and {key2} are statistically different (Bootstrapping).")
            else:
                print(f"There is no significant difference between the distributions of {key1} and {key2} (Bootstrapping).")

            # Convert the tuple keys to strings before updating the p_value_dict
            str_key1 = ', '.join(matching_criteria_1)

            # Update the p_value_dict with the p-values using the selected keys
            p_value_dict[str_key1] = {
                'p_value_ttest': round(p_value_ttest, 2),
                'p_value_welchsttest': round(p_value_welch, 2),
                'bootstrap_p_value': round(bootstrap_p_value, 2)
            }

            # Save the updated p_value_dict back to the JSON file
            with open(p_value_file, 'w') as file:
                json.dump(p_value_dict, file)
        else:
            print("No auto-filled options available based on Key1.")
    except ValueError:
        print("Invalid input for Key1. Please enter a valid number.")
else:
    print("No keys without 'Sham' available in the dictionary.")


Available keys without 'Sham' (showing first 3 elements):
1. D1 Right Turn Contralateral Right
2. D1 Right Turn Contralateral Locomotion
3. D1 Right Turn Contralateral Face
4. D1 Right Turn Bilateral Right
5. D1 Right Turn Bilateral Locomotion
6. D1 Right Turn Bilateral Face
7. D1 Right Turn Ipsilateral Right
8. D1 Right Turn Ipsilateral Locomotion
9. D1 Right Turn Ipsilateral Face
10. D1 Locomotion Right
11. D1 Locomotion Locomotion
12. D1 Locomotion Face
13. D1 Face Groom Right
14. D1 Face Groom Locomotion
15. D1 Face Groom Face
Enter the number corresponding to Key1: 15
You have selected Key1: D1 Face Groom Face Groom Bouts Stim Bock
['D1', 'Face', 'Groom', 'Face', 'Groom', 'Bouts', 'Stim', 'Bock']
Auto-filled options for Key2:
1. D1 Face Groom Bouts
Auto-filled Key2 based on matching criteria: D1 Face Groom Bouts Sham
T-Test Results:
t-statistic (T-Test): -0.3565206633248731
p-value (T-Test): 0.7248472551203001
Welch's T-Test Results:
t-statistic (Welch's T-Test): -0.26799063962256

In [287]:
# Define a function to determine two different keys based on conditions
def determine_key(selected_header, unique_header_3_value, unique_header_4_value,unique_header_0_value,unique_header_10_value, unique_header_1_value,unique_header_6_value):
    # Split the selected_header by spaces and enumerate the strings
    header_parts = selected_header.split()
    print(header_parts)
    for i, part in enumerate(header_parts):
        if part == "Array":
            if i >= 2:
                new_selected_header = f"{header_parts[i - 2]} {header_parts[i - 1]} Duration"
            else:
                new_selected_header = f"{header_parts[i - 1]} Duration"

        # Initialize two keys with common components
    key_1 = f"{unique_header_1_value} {unique_header_3_value} {new_selected_header} Stim"
    key_2 = f"{unique_header_1_value} {unique_header_3_value} {new_selected_header} Post"

    # Check if unique_header_4_value is equal to "y" for both keys
    if unique_header_4_value == "y":
        key_1 = f"{unique_header_1_value} {new_selected_header} Stim Sham"
        key_2 = f"{unique_header_1_value} {new_selected_header} Post Sham"
    else:
        # Check if "Right Turn" is in unique_header_3_value for key_1
        if "Right Turn" in unique_header_3_value:
            key_1 = f"{unique_header_1_value} {unique_header_3_value} {unique_header_6_value} {new_selected_header} Stim"

        # Check if "Right Turn" is in unique_header_3_value for key_2
        if "Right Turn" in unique_header_3_value:
            key_2 = f"{unique_header_1_value} {unique_header_3_value} {unique_header_6_value} {new_selected_header} Post"

    return key_1, key_2

# Initialize the averages_dict
averages_dict = {}

# Load the existing averages_dict from the file if it exists
averages_dict_file = "Duration practice_dict_file"
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    averages_dict = {}

# Get a list of unique headers
unique_headers = matching_rows.columns.tolist()  # Get all available columns as headers

# Uncomment one of the following lines based on your choice:

# Option 1: Specify header indices manually
header_indices_to_process = [7, 15, 23]

# Option 2: Choose a single header using user input
#print("\nAvailable headers:")
#for idx, header in enumerate(matching_rows.columns):
    #print(f"{idx}: {header}")
#header_selection = int(input("Enter the number of the selected header: "))
#header_indices_to_process = [header_selection] if 'header_selection' in locals() else []

# Initialize key1 and key2 with empty strings
key1 = ""
key2 = ""

# Iterate over the selected header indices
for header_selection in header_indices_to_process:
    selected_header = matching_rows.columns[header_selection]

    # Filter the DataFrame to select all rows with the selected header
    filtered_rows = matching_rows[selected_header]

    # Set the values for unique_header_1_value, unique_header_3_value, and unique_header_4_value
    unique_header_1_value = matching_rows.iloc[0, 1]
    unique_header_3_value = matching_rows.iloc[0, 3]
    unique_header_4_value = matching_rows.iloc[0, 4]
    unique_header_6_value = matching_rows.iloc[0, 6]
    unique_header_0_value = "Stim"
    unique_header_10_value = "Post"

    # Initialize lists to store average stim and post durations for each row
    average_stim_durations = []
    average_post_durations = []

    # Process the filtered data for both "stim" and "post"
    for row_index, row in filtered_rows.items():
        # Initialize lists for durations in
        # the current row for both "stim" and "post"
        stim_durations = []
        post_durations = []

        # Remove the additional square brackets at the beginning and end of the row
        row = row.strip('[]')

        # Split the row into individual tuples using '][' as the delimiter
        tuples_list = row.split('][')

        # Iterate over the tuples in the list
        for tuple_str in tuples_list:
            # Remove the square brackets and split the tuple values by comma
            values = [float(val) for val in tuple_str.strip('[]').split(',')]

            # Now, 'values' contains the individual values as floats
            # You can access and manipulate the values based on your conditions
            if len(values) >= 7:
                value5 = values[5]
                value6 = values[6]

                # Apply conditions for "stim" analysis
                if value5 > 3000 and value6 <= 21000:
                    stim_durations.append(values[4])  # Append the duration value to the stim_durations list
                # Apply conditions for "post" analysis
                if value5 > 21000 and value6 <= 39000:
                    post_durations.append(values[4])

        # Calculate the average "stim" and "post" durations for the current row
        average_stim_duration = sum(stim_durations) / len(stim_durations) if stim_durations else None
        average_post_duration = sum(post_durations) / len(post_durations) if post_durations else None

        # Append the average durations to the respective lists
        average_stim_durations.append(average_stim_duration)
        average_post_durations.append(average_post_duration)

        # Print the calculated averages for the current row
        #print(f"Row {row_index}: Average Stim Duration: {average_stim_duration}, Average Post Duration: {average_post_duration}")

    # Determine the key for the current row
    key1, key2 = determine_key(selected_header, unique_header_3_value, unique_header_4_value, unique_header_0_value, unique_header_10_value, unique_header_1_value, unique_header_6_value)
    print(key1)
    print(key2)

    # Create a dictionary to hold the durations for the current row
    duration_data = {
        'stim_duration': average_stim_durations,
        'post_duration': average_post_durations
    }
    # Print the duration_data dictionary
    #print("Duration Data:")
    #print(duration_data)

    # Use key1 and key2 when storing values in the averages_dict
    if "Stim" in key1:
        # Check if 'Stim' key exists in averages_dict, create if not
        if 'Stim' not in averages_dict:
            averages_dict['Stim'] = {}
        averages_dict['Stim'][key1] = duration_data['stim_duration']

    if "Post" in key2:
        # Check if 'Post' key exists in averages_dict, create if not
        if 'Post' not in averages_dict:
            averages_dict['Post'] = {}
        averages_dict['Post'][key2] = duration_data['post_duration']
        
    # Print the Averages Dictionary
    #print("Averages Dictionary:")
    #print(averages_dict)
try:
    with open(averages_dict_file, 'w') as file:
        json.dump(averages_dict, file, separators=(',', ':'))
except Exception as e:
    print(f"An error occurred while saving the file: {e}")

['Right', 'Turn', 'Array']
D1 Locomotion Right Turn Duration Stim
D1 Locomotion Right Turn Duration Post
['Locomotion', 'Array']
D1 Locomotion Locomotion Duration Stim
D1 Locomotion Locomotion Duration Post
['Face', 'Groom', 'Array']
D1 Locomotion Face Groom Duration Stim
D1 Locomotion Face Groom Duration Post


In [288]:
# Define the path to the JSON file where you want to store the dictionary
p_value_file = "p_value_inner_trial.json"
averages_dict_file = "Duration practice_dict_file"

# Load data from the JSON file
try:
    with open(averages_dict_file, 'r') as file:
        data_dict = json.load(file)
except FileNotFoundError:
    data_dict = {}

# Initialize the p_value_dict as a global variable or load it from the JSON file if it exists
try:
    with open(p_value_file, 'r') as file:
        p_value_dict = json.load(file)
except FileNotFoundError:
    p_value_dict = {}

# Initialize the lists for 'Post' and 'Stim' keys
keys_post = []
keys_stim = []

# Iterate through the sub-dictionaries 'pos' and 'stim'
for sub_dict_name in ['Post', 'Stim']:
    sub_dict = data_dict.get(sub_dict_name, {})  # Get the sub-dictionary

    # Filter 'Post' keys and 'Stim' keys without 'Sham'
    post_keys = [key for key in sub_dict.keys() if 'Post' in key and 'Sham' not in key]
    stim_keys = [key for key in sub_dict.keys() if 'Stim' in key and 'Sham' not in key]

    # Extend the main lists with the filtered keys
    keys_post.extend(post_keys)
    keys_stim.extend(stim_keys)

# Check if there are 'Post' keys without 'Sham' to use as Key1
if keys_post:
    print("Available 'Post' keys without 'Sham' (showing first 3 elements):")
    for idx, key in enumerate(keys_post, start=1):
        # Split the key using space as the separator
        key_elements = key.split(' ')

        # Check if element 2 is "Right" and combine it with elements 3 and 4
        if len(key_elements) >= 4 and key_elements[1] == 'Right':
            formatted_key = ' '.join([key_elements[0], key_elements[1] + ' ' + key_elements[2] + ' ' + key_elements[3],key_elements[4]])
        elif len(key_elements)>= 3 and key_elements[1]== 'Face':
            formatted_key = ' '.join([key_elements[0], key_elements[1] + ' ' + key_elements[2],key_elements[3]])
        else:
            formatted_key = ' '.join(key_elements[:3])

        print(f"{idx}. {formatted_key}")

    # Prompt the user to select Key1
    try:
        selected_key_idx1 = int(input("Enter the number corresponding to Key1: ")) - 1
        key1 = keys_post[selected_key_idx1]
        print(f"You have selected Key1: {key1}")

     # Auto-fill Key2 based on the matching criteria
        matching_criteria_1 = key1.split(' ')
        if matching_criteria_1[1] =='Locomotion' and matching_criteria_1[2] =='Locomotion':
             matching_criteria = [matching_criteria_1[0], matching_criteria_1[1], matching_criteria_1[2]]
        else:
            matching_criteria = [matching_criteria_1[0], matching_criteria_1[1], matching_criteria_1[2], matching_criteria_1[3], matching_criteria_1[4]]

        # Check if matching_criteria_1[1] and matching_criteria_1[2] are 'Locomotion'
        if matching_criteria_1[1] == 'Locomotion' and matching_criteria_1[2] == 'Locomotion':
            auto_fill_keys = [key for key in keys_stim if matching_criteria == key.split(' ')[:3]]
        else:
            auto_fill_keys = [key for key in keys_stim if matching_criteria == key.split(' ')[:5]]

        if auto_fill_keys:
            print("Auto-filled options for Key2:")
            for idx, key in enumerate(auto_fill_keys[:5], start=1):
                # Extract the first three elements of the key
                key_elements = key.split(' ')[:4]
                formatted_key = ' '.join(key_elements)
                print(f"{idx}. {formatted_key}")

            # Automatically set Key2 based on the matching criteria
            key2 = auto_fill_keys[0]  # You can adjust this if you have specific logic to choose among them
            print(f"Auto-filled Key2 based on matching criteria: {key2}")

            # Retrieve the samples associated with the selected keys
            samples1 = data_dict['Post'].get(key1, [])
            samples2 = data_dict['Stim'].get(key2, [])

            # Perform bootstrapping with 10000 iterations
            num_bootstrap_samples = 10000
            bootstrap_mean_differences = []

            # Perform bootstrapping with 'num_bootstrap_samples' iterations
            for _ in range(num_bootstrap_samples):
                # Generate bootstrap samples by resampling with replacement
                bootstrap_samples1 = np.random.choice(samples1, len(samples1), replace=True)
                bootstrap_samples2 = np.random.choice(samples2, len(samples2), replace=True)

                # Calculate the mean difference between bootstrap_samples1 and bootstrap_samples2
                bootstrap_mean_difference = np.mean(bootstrap_samples1) - np.mean(bootstrap_samples2)

                # Append the mean difference to the list
                bootstrap_mean_differences.append(bootstrap_mean_difference)

            # Calculate the observed difference in means between the original samples
            observed_diff = np.mean(samples1) - np.mean(samples2)

            # Calculate the p-value for the bootstrap test
            bootstrap_p_value = (np.sum(np.abs(bootstrap_mean_differences) >= np.abs(observed_diff)) + 1) / (
                num_bootstrap_samples + 1)

            # Perform a t-test on the original samples
            t_statistic, p_value = stats.ttest_ind(samples1, samples2)

            # Print the t-test results
            print("T-Test Results:")
            print("t-statistic:", t_statistic)
            print("p-value (T-Test):", p_value)

            # Print the bootstrapping results
            print("Bootstrapping Results:")
            print("Bootstrap p-value:", bootstrap_p_value)

            # Determine whether the t-test results are statistically significant (typically p < 0.05)
            if round(p_value, 2) <= 0.05:
                print(f"The means of {key1} and {key2} are statistically different.")
            else:
                print(f"There is no significant difference between the means of {key1} and {key2}.")

            # Convert the tuple keys to strings before updating the p_value_dict
                 
            if len(matching_criteria_1) >= 4 and matching_criteria_1[1] == 'Right':
                str_key1 = ' '.join([matching_criteria_1[1] + ' ' + matching_criteria_1[2] + ' ' + matching_criteria_1[3],matching_criteria_1[4]])
            elif len(matching_criteria_1)>= 3 and matching_criteria_1[1]== 'Face':
                str_key1 = ' '.join([key_elements[0], matching_criteria_1[1] + ' ' + matching_criteria_1[2],matching_criteria_1[3]])
            else:
                str_key1 = ' '.join(matching_criteria_1[:3])


            # Update the p_value_dict with the p-values using the selected keys
            p_value_dict[str_key1] = {
                'p_value': round(p_value, 2),
                'bootstrap_p_value': round(bootstrap_p_value, 2)
            }

            # Save the updated p_value_dict back to the JSON file
            with open(p_value_file, 'w') as file:
                json.dump(p_value_dict, file)
        else:
            print("No auto-filled options available based on Key1.")
    except ValueError:
        print("Invalid input for Key1. Please enter a valid number.")
else:
    print("No 'Post' keys without 'Sham' available in the dictionary.")


Available 'Post' keys without 'Sham' (showing first 3 elements):
1. D1 Locomotion Right
2. D1 Locomotion Locomotion
3. D1 Locomotion Face
Enter the number corresponding to Key1: 1
You have selected Key1: D1 Locomotion Right Turn Duration Post
Auto-filled options for Key2:
1. D1 Locomotion Right Turn
Auto-filled Key2 based on matching criteria: D1 Locomotion Right Turn Duration Stim
T-Test Results:
t-statistic: -0.11970852921398406
p-value (T-Test): 0.9070848800908163
Bootstrapping Results:
Bootstrap p-value: 0.8958104189581042
There is no significant difference between the means of D1 Locomotion Right Turn Duration Post and D1 Locomotion Right Turn Duration Stim.


In [133]:
import re

# Read data from the JSON file
with open('p_value_inner_trial.json', 'r') as json_file:
    data = json.load(json_file)

# Create dictionaries to store p-values and labels
p_value_dict = {}
row_labels = set()
column_labels = set()

# Define a function to normalize labels to lowercase
def normalize_label(label):
    return label.lower()

# Extract and organize the p-values, row labels, and column labels
for key, p_values in data.items():
    elements = key.split(" ")
    if elements[1]=='Turn': 
        elements[1]= [elements[0] +" "+ elements[1]+" "+elements[2]]
        elements=elements[1]
    elif elements[1]=='Face':
        elements=[elements[0],elements[1]+" "+elements[2],elements[3]]
    
    print(elements)
    if len(elements) >= 3:
        row_label = normalize_label(elements[1])  # Normalize to lowercase
        col_label = normalize_label(elements[2])  # Normalize to lowercase
        
        row_labels.add(row_label)
        column_labels.add(col_label)
        
        p_value_dict[(row_label, col_label)] = p_values.get("p_value_ttest")

# Sort the labels alphabetically
sorted_row_labels = sorted(row_labels)
sorted_column_labels = sorted(column_labels)

# Create a list to hold the data in a tabular format
table_data = []

# Define a function to add color to the cell box based on significance
def color_box(p_value):
    if isinstance(p_value, float):
        if p_value < 0.01:
            return f"\033[41m{p_value:.2f}\033[0m"  # Red background for highly significant
        elif p_value < 0.05:
            return f"\033[43m{p_value:.2f}\033[0m"  # Yellow background for moderately significant
        else:
            return f"{p_value:.2f}"  # Default background for non-significant p-values
    else:
        return p_value

# Build the table data with colored cell boxes
for row_label in sorted_row_labels:
    row = [row_label]
    for col_label in sorted_column_labels:
        p_value = p_value_dict.get((row_label, col_label), '')
        colored_box = color_box(p_value)
        row.append(colored_box)
    table_data.append(row)

# Print the Punnett square-like structure with improved readability and colored cell boxes
table_headers = [""] + sorted_column_labels
table = tabulate(table_data, headers=table_headers, tablefmt="fancy_grid", numalign="center", stralign="center")

print(table)

['D1', 'Locomotion', 'Right']
['D1', 'Locomotion', 'Locomotion']
['D1', 'Locomotion', 'Face']
['D1', 'Face Groom', 'Right']
['D1', 'Face Groom', 'Locomotion']
['D1', 'Face Groom', 'Face']
['Right', 'Turn', 'Bilateral', 'Right']
['Right', 'Turn', 'Bilateral', 'Locomotion']
['Right', 'Turn', 'Bilateral', 'Face']
['Right', 'Turn', 'Contralateral', 'Right']
['Right', 'Turn', 'Contralateral', 'Locomotion']
['Right', 'Turn', 'Contralateral', 'Face']
['Right', 'Turn', 'Ipsilateral', 'Right']
['Right', 'Turn', 'Ipsilateral', 'Locomotion']
['Right', 'Turn', 'Ipsilateral', 'Face']
╒════════════╤═════════════╤═════════════════╤════════╤═══════════════╤══════════════╤═════════╕
│            │  bilateral  │  contralateral  │  face  │  ipsilateral  │  locomotion  │  right  │
╞════════════╪═════════════╪═════════════════╪════════╪═══════════════╪══════════════╪═════════╡
│ face groom │             │                 │        │               │              │         │
├────────────┼─────────────┼──────

## Using Matching Tuples

In [358]:
averages_dict_file = "averages_dict_file"

# Try to open the file for reading, if it exists
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    averages_dict = {}

# Initialize dictionaries for stim_count and duration
bout_count = {}
duration = {}

for row_index, tuples_list in matching_tuples.items():
    # Initialize dictionaries for each row index
    bout_count[row_index] = []
    duration[row_index] = []

    for matching_tuple in tuples_list:
        if len(matching_tuple) > 3:
            bout_count[row_index].append(matching_tuple[3])
        if len(matching_tuple) > 4:
            duration[row_index].append(matching_tuple[4])

# Calculate totals for bout_count and duration for each row index
total_bout_count = {}
total_duration = {}

for row_index in matching_tuples.keys():
    total_bout_count[row_index] = len(bout_count[row_index])
    total_duration[row_index] = sum(duration[row_index])

# Extract total bout count values as a list
total_bout_count_values = list(total_bout_count.values())
average_total_bout_count = round((sum(total_bout_count_values) / len(total_bout_count_values)), 2)

total_duration_values = list(total_duration.values())
average_total_duration = round((sum(total_duration_values) / len(total_duration_values)), 2)

# Update the averages_dict with the calculated averages
# Convert tuple keys to strings and save with double quotes
for i in range(len(filtering_criteria) - 1):  # Iterate up to the second-to-last element
    if filtering_criteria[i] == "Selected Header: Sham" and filtering_criteria[i + 1] == "Filter Value: y":
        averages_dict["D1, Face Groom Array, Bout, Sham"] = total_bout_count_values
        averages_dict["D1, Face Groom Array, Duration, Sham"] = total_duration_values
        break  # Exit the loop if the condition is met

# If the loop completes without finding the condition, update the other keys
else:
    averages_dict["D1, Locomotoion, Face Groom Array, Bout"] = total_bout_count_values
    averages_dict["D1, Locomotion, Face Groom Array, Duration"] = total_duration_values

# Save the updated averages_dict back to the file with double quotes
with open(averages_dict_file, 'w') as file:
    json.dump(averages_dict, file)

# Print the dictionary
#print("Averages Dictionary:")
#print(averages_dict)

# Print the average
print("Average Total Bout Count:", average_total_bout_count)
print("Average Total Duration Count:", average_total_duration)

# Print totals for stim_count and duration for each row index
#for row_index in matching_tuples.keys():
    #print(f'Totals for Row Index {row_index}:')
    #print('Total Bout Count:', total_bout_count[row_index])
    #print('Total Duration:', total_duration[row_index])
    #print()



Average Total Bout Count: 254.67
Average Total Duration Count: 1876.33


In [360]:
# Define the path to the JSON file where you want to store the dictionary
p_value_file = "p_value_file.json"

# Initialize the p_value_dict as a global variable or load it from the JSON file if it exists
try:
    with open(p_value_file, 'r') as file:
        p_value_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    p_value_dict = {}

# Initialize the averages_dict as a global variable or load it from the JSON file if it exists
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # Handle the file not found error appropriately
    averages_dict = {}

# Extract keys without 'Sham' and keys with 'Sham' from averages_dict
keys_without_sham = [key for key in averages_dict.keys() if 'Sham' not in key]
keys_with_sham = [key for key in averages_dict.keys() if 'Sham' in key]

# Check if there are keys without 'Sham' to use as Key1
if keys_without_sham:
    print("Available keys without 'Sham' (showing first 3 elements):")
    for idx, key in enumerate(keys_without_sham, start=1):
        # Extract the first three elements of the key
        key_elements = key.split(', ')
        formatted_key = ', '.join(key_elements)
        print(f"{idx}. {formatted_key}")

    # Prompt the user to select Key1
    try:
        selected_key_idx1 = int(input("Enter the number corresponding to Key1: ")) - 1
        key1 = keys_without_sham[selected_key_idx1]
        print(f"You have selected Key1: {key1}")

        # Auto-fill Key2 based on the matching criteria
        matching_criteria_1 = key1.split(', ')
        matching_criteria = [matching_criteria_1[0],matching_criteria_1[2], matching_criteria_1[3]]
        print(matching_criteria)
        auto_fill_keys = [key for key in keys_with_sham if
                          matching_criteria == key.split(', ')[:3]]

        if auto_fill_keys:
            print("Auto-filled options for Key2:")
            for idx, key in enumerate(auto_fill_keys[:3], start=1):
                # Extract the first three elements of the key
                key_elements = key.split(', ')[:3]
                formatted_key = ', '.join(key_elements)
                print(f"{idx}. {formatted_key}")

            # Automatically set Key2 based on the matching criteria
            key2 = auto_fill_keys[0]  # You can adjust this if you have specific logic to choose among them
            print(f"Auto-filled Key2 based on matching criteria: {key2}")

            # Retrieve the samples associated with the selected keys
            samples1 = averages_dict.get(key1, [])
            samples2 = averages_dict.get(key2, [])
            

            # Perform bootstrapping with 1000 iterations
            num_bootstrap_samples = 10000
            bootstrap_mean_differences = []

            # Perform bootstrapping with 'num_bootstrap_samples' iterations
            for _ in range(num_bootstrap_samples):
                # Generate bootstrap samples by resampling with replacement
                bootstrap_samples1 = np.random.choice(samples1, len(samples1), replace=True)
                bootstrap_samples2 = np.random.choice(samples2, len(samples2), replace=True)

                # Calculate the mean difference between bootstrap_samples1 and bootstrap_samples2
                bootstrap_mean_difference = np.mean(bootstrap_samples1) - np.mean(bootstrap_samples2)

                # Append the mean difference to the list
                bootstrap_mean_differences.append(bootstrap_mean_difference)

            # Calculate the observed difference in means between the original samples
            observed_diff = np.mean(samples1) - np.mean(samples2)

            # Calculate the p-value for the bootstrap test
            bootstrap_p_value_ttest = (np.sum(np.abs(bootstrap_mean_differences) >= np.abs(observed_diff)) + 1) / (
                num_bootstrap_samples + 1)
            
            # Perform a t-test on the original samples
            t_statistic_ttest, p_value_ttest = stats.ttest_ind(samples1, samples2)

           
            # Perform Welch's t-test
            t_statistic_welch, p_value_welch = stats.ttest_ind(samples1, samples2, equal_var=False)
            
            # Print the t-test results
            print("T-Test Results:")
            print("t-statistic:", t_statistic_ttest)
            print("p-value (T-Test):", p_value_ttest)

            # Print the Welch's T-Test Results
            print("Welch's T-Test Results:")
            print("t-statistic (Welch's T-Test):", t_statistic_welch)
            print("p-value (Welch's T-Test):", p_value_welch)


            # Print the bootstrapping results
            print("Bootstrapping Results:")
            print("Bootstrap p-value:", bootstrap_p_value)

            # Determine whether the t-test results are statistically significant (typically p < 0.05)
            if round(p_value_ttest,2) <= 0.05:
                print(f"The means of {key1} and {key2} are statistically different (T-Test).")
            else:
                print(f"There is no significant difference between the means of {key1} and {key2} (T-Test).")

            # Determine whether the Mann-Whitney U test results are statistically significant (typically p < 0.05)
            if round(bootstrap_p_value_ttest,2) <= 0.05:
                print(f"The distributions of {key1} and {key2} are statistically different (Welch's T Test.")
            else:
                print(f"There is no significant difference between the distributions of {key1} and {key2} (Welchs T Test).")

            # Determine whether the bootstrapping results are statistically significant (typically p < 0.05)
            if round(bootstrap_p_value,2) <= 0.05:
                print(f"The means of {key1} and {key2} are statistically different (Bootstrapping).")
            else:
                print(f"There is no significant difference between the means of {key1} and {key2} (Bootstrapping).")

            # Convert the tuple keys to strings before updating the p_value_dict
            str_key1 = ', '.join(matching_criteria_1)

            # Update the p_value_dict with the p-values using the selected keys
            p_value_dict[str_key1] = {
                'p_value_ttest': round(p_value_ttest,2),
                'p_value_welchsttest': round(p_value_welch,2),
                'bootstrap_p_value': round(bootstrap_p_value,2)
            }

            # Save the updated p_value_dict back to the JSON file
            with open(p_value_file, 'w') as file:
                json.dump(p_value_dict, file)
        else:
            print("No auto-filled options available based on Key1.")
    except ValueError:
        print("Invalid input for Key1. Please enter a valid number.")
else:
    print("No keys without 'Sham' available in the dictionary.")

Available keys without 'Sham' (showing first 3 elements):
1. D1, Right Turn, Right Turn Array, Bout
2. D1, Right Turn, Right Turn Array, Duration
3. D1, Right Turn, Locomotion Array, Bout
4. D1, Right Turn, Locomotion Array, Duration
5. D1, Right Turn, Face Groom Array, Bout
6. D1, Right Turn, Face Groom Array, Duration
7. D1, Right Turn-Contralateral, Right Turn Array, Bout
8. D1, Right Turn-Contralateral, Right Turn Array, Duration
9. D1, Right Turn-Contralateral, Locomotion Array, Bout
10. D1, Right Turn-Contralateral, Locomotion Array, Duration
11. D1, Right Turn-Contralateral, Face Groom Array, Bout
12. D1, Right Turn-Contralateral, Face Groom Array, Duration
13. D1, Right Turn-Ipsilateral, Right Turn Array, Bout
14. D1, Right Turn-Ipsilateral, Right Turn Array, Duration
15. D1, Right Turn-Ipsilateral, Locomotion Array, Bout
16. D1, Right Turn-Ipsilateral, Locomotion Array, Duration
17. D1, Right Turn-Ipsilateral, Face Groom Array, Bout
18. D1, Right Turn-Ipsilateral, Face Groom A

In [160]:
import re

# Read data from the JSON file
with open('p_value_file.json', 'r') as json_file:
    data = json.load(json_file)

# Create dictionaries to store p-values and labels
p_value_dict = {}
row_labels = set()
column_labels = set()

# Define a function to normalize labels to lowercase
def normalize_label(label):
    return label.lower()

# Extract and organize the p-values, row labels, and column labels
for key, p_values in data.items():
    elements = re.split(r',\s*', key)
    if len(elements) >= 3:
        row_label = normalize_label(elements[1])  # Normalize to lowercase
        col_label = normalize_label(elements[2])  # Normalize to lowercase
        
        row_labels.add(row_label)
        column_labels.add(col_label)
        
        p_value_dict[(row_label, col_label)] = p_values.get("p_value_ttest")

# Sort the labels alphabetically
sorted_row_labels = sorted(row_labels)
sorted_column_labels = sorted(column_labels)

# Create a list to hold the data in a tabular format
table_data = []

# Define a function to add color to the cell box based on significance
def color_box(p_value):
    if isinstance(p_value, float):
        if p_value < 0.01:
            return f"\033[41m{p_value:.2f}\033[0m"  # Red background for highly significant
        elif p_value < 0.05:
            return f"\033[43m{p_value:.2f}\033[0m"  # Yellow background for moderately significant
        else:
            return f"{p_value:.2f}"  # Default background for non-significant p-values
    else:
        return p_value

# Build the table data with colored cell boxes
for row_label in sorted_row_labels:
    row = [row_label]
    for col_label in sorted_column_labels:
        p_value = p_value_dict.get((row_label, col_label), '')
        colored_box = color_box(p_value)
        row.append(colored_box)
    table_data.append(row)

# Print the Punnett square-like structure with improved readability and colored cell boxes
table_headers = [""] + sorted_column_labels
table = tabulate(table_data, headers=table_headers, tablefmt="fancy_grid", numalign="center", stralign="center")

print(table)

╒══════════════════════════╤════════════════════╤════════════════════╤════════════════════╕
│                          │  face groom array  │  locomotion array  │  right turn array  │
╞══════════════════════════╪════════════════════╪════════════════════╪════════════════════╡
│        face groom        │        0.54        │        0.03        │        0.31        │
├──────────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│        locomotion        │        0.58        │        0.72        │        0.93        │
├──────────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│        right turn        │        0.93        │        0.89        │        0.92        │
├──────────────────────────┼────────────────────┼────────────────────┼────────────────────┤
│ right turn-contralateral │        0.04        │        0.07        │        0.42        │
├──────────────────────────┼────────────────────┼────────────────────┼──────────

In [151]:
averages_dict_file = "averages_stim_dict_file"

# Try to open the file for reading, if it exists
try:
    with open(averages_dict_file, 'r') as file:
        averages_dict = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    averages_dict = {}

# Initialize dictionaries for stim_count and duration
bout_count = {}
duration = {}

for row_index, tuples_list in matching_tuples.items():
    # Initialize dictionaries for each row index
    bout_count[row_index] = []
    duration[row_index] = []

    for matching_tuple in tuples_list:
        if matching_tuple[5]>3000 and matching_tuple[6]<=21000:
            if len(matching_tuple) > 3:
                bout_count[row_index].append(matching_tuple[3])
            if len(matching_tuple) > 4:
                duration[row_index].append(matching_tuple[4])

# Calculate totals for bout_count and duration for each row index
total_bout_count = {}
total_duration = {}

for row_index in matching_tuples.keys():
    total_bout_count[row_index] = len(bout_count[row_index])
    total_duration[row_index] = sum(duration[row_index])
    
print("Total Bout Count:")
for row_index, bout_count_total in total_bout_count.items():
    print(f"Row Index {row_index}: {bout_count_total}")

print("\nTotal Duration:")
for row_index, duration_total in total_duration.items():
    print(f"Row Index {row_index}: {duration_total}")
    
    
# Extract total bout count values as a list
total_bout_count_values = list(total_bout_count.values())
average_total_bout_count = round((sum(total_bout_count_values) / len(total_bout_count_values)), 2)

total_duration_values = list(total_duration.values())
average_total_duration = round((sum(total_duration_values) / len(total_duration_values)), 2)

# Update the averages_dict with the calculated averages
# Convert tuple keys to strings and save with double quotes
for i in range(len(filtering_criteria) - 1):  # Iterate up to the second-to-last element
    if filtering_criteria[i] == "Selected Header: Sham" and filtering_criteria[i + 1] == "Filter Value: y":
        averages_dict["D1, Face Groom Array, Bout, Sham"] = total_bout_count_values
        averages_dict["D1, Face Groom Array, Duration, Sham"] = total_duration_values
        break  # Exit the loop if the condition is met

# If the loop completes without finding the condition, update the other keys
else:
    averages_dict["D1, Locomotoion, Locomotionn Array, Bout"] = total_bout_count_values
    averages_dict["D1, Locomotion, Locomotion Array, Duration"] = total_duration_values

# Save the updated averages_dict back to the file with double quotes
with open(averages_dict_file, 'w') as file:
    json.dump(averages_dict, file)

# Print the dictionary
#print("Averages Dictionary:")
#print(averages_dict)

# Print the average
print("Average Total Bout Count:", average_total_bout_count)
print("Average Total Duration Count:", average_total_duration)

# Print totals for stim_count and duration for each row index
#for row_index in matching_tuples.keys():
    #print(f'Totals for Row Index {row_index}:')
    #print('Total Bout Count:', total_bout_count[row_index])
    #print('Total Duration:', total_duration[row_index])
    #print()



Total Bout Count:
Row Index 3: 40
Row Index 4: 80

Total Duration:
Row Index 3: 170
Row Index 4: 358
Average Total Bout Count: 60.0
Average Total Duration Count: 264.0


In [291]:
# Check if a JSON file containing plotted_data exists
try:
    with open("plotted_data.json", "r") as file:
        plotted_data = json.load(file)
except FileNotFoundError:
    # If the file doesn't exist, initialize an empty dictionary
    plotted_data = {}
# Assuming 'matching_tuples' contains the selected tuples
# and 'time_values' contains the corresponding time values

# Define the selected indices for start and end time
start_time_index = 5
end_time_index = 6
bin_width_minutes =5
# Define a dictionary to map index values to names
index_name_mapping = {
    0: 'Behavior',
    1: 'Stim Count',
    2: 'Frame Rate',
    3: 'Bouts',
    4: 'Duration',
    5: 'Start Time',
    6: 'End Time'
}

# Extract the name associated with the selected index
selected_index_name = index_name_mapping[selected_index]

selected_values = [tuple[selected_index]for tuple in values for row_index, values in matching_tuples.items()]

# Extract the start and end times from matching_tuples and convert to minutes
start_times = [tuple[start_time_index] / 600 for values in matching_tuples.values() for tuple in values]
end_times = [tuple[end_time_index] / 600 for values in matching_tuples.values() for tuple in values]


# Calculate the minimum and maximum times
min_time = 0
max_time = 65

# Create time bins
time_bins = np.arange(min_time, max_time + bin_width_minutes, bin_width_minutes)

# Initialize lists to store mean values and SEM
mean_values = []
sem_values = []


# Population size (N) is the total number of data points in your entire dataset
N = len(matching_tuples)

# Calculate the population average and standard deviation for the entire dataset
population_average = np.mean(selected_values)
population_std = np.std(selected_values, ddof=1)

# Print the population information as a single line with the selected index name
print(f"Selected Index Name: {selected_index_name}, Population Size (N): {N}, Population Average: {population_average}, Population Std Deviation: {population_std}")


# Integrate the new functionality to prompt the user for binning
user_choice = input("Do you want to graph the selected index with per minute with a specific bin length for each key? (yes/no): ")

if user_choice.lower() == "yes":

    # Define the selected bin width in minutes
    bin_width_minutes = float(input("Enter the bin width in minutes: "))
    
    # Initialize dictionaries to store mean values and SEM for each key
    key_mean_values = {}
    key_sem_values = {}
    # Initialize a dictionary to store bin counts for each key
    key_bin_counts = {}

    # Loop through each key in the 'matching_tuples' dictionary
    for key, values in matching_tuples.items():
        # Extract the selected index name associated with the key
        selected_index_name = index_name_mapping[selected_index]

        # Extract the selected values at the selected index for the current key
        selected_values = [tuple[selected_index] for tuple in values]

        # Extract the start and end times for the current key and convert to minutes
        start_times = [tuple[5] / 600 for tuple in values]
        end_times = [tuple[6] / 600 for tuple in values]

        # Create time bins
        time_bins = np.arange(min_time, max_time + bin_width_minutes, bin_width_minutes)

        # Initialize dictionaries to store bin values and bin start/end times
        bin_data = {
            'Bin Start Time': [],
            'Bin End Time': [],
            'Selected Values': [],
            'Bin Values': []
        }

        # Initialize a list to store bin counts for the current key
        bin_counts = []

        # Iterate through the time bins for the current key
        for bin_start, bin_end in zip(time_bins[:-1], time_bins[1:]):
            bin_values = []

            # Iterate through the data to collect values within this bin
            for start_time, end_time, value_at_selected_index in zip(start_times, end_times, selected_values):
                if start_time >= bin_start and end_time <= bin_end:
                    # Value at the selected index falls within this bin for the current key
                    bin_values.append(value_at_selected_index)

            # Append data to the bin_data dictionary for printing
            bin_data['Bin Start Time'].append(bin_start)
            bin_data['Bin End Time'].append(bin_end)
            bin_data['Selected Values'].append(selected_values)
            bin_data['Bin Values'].append(bin_values)

            # Calculate the number of elements in this bin and store it in bin_counts
            bin_count = len(bin_values)
            bin_counts.append(bin_count)

        # Store the bin counts for the current key in the key_bin_counts dictionary
        key_bin_counts[key] = bin_counts

    # Initialize a dictionary to store the result
    bin_counts_per_minute = {}

    # Iterate through the keys and their corresponding bin counts
    for key, bin_counts in key_bin_counts.items():
        # Divide each bin count by bin_width_in_minutes
        bin_counts_per_minute[key] = [count / bin_width_minutes for count in bin_counts]

    # Initialize a list to store the counts per minute for each bin across all keys
    all_bin_counts = [[] for _ in range(len(bin_counts_per_minute[next(iter(bin_counts_per_minute))]))]

    # Populate the list with counts per minute from all keys for each bin
    for counts_per_minute in bin_counts_per_minute.values():
        for i, count in enumerate(counts_per_minute):
            all_bin_counts[i].append(count)

    # Initialize lists to store the bin mean and SEM
    bin_means = []
    bin_sem = []

    # Calculate the mean and SEM for each bin across all keys
    for bin_counts in all_bin_counts:
        bin_mean = np.mean(bin_counts)  # Calculate the mean for the bin
        bin_means.append(bin_mean)  # Store the bin mean in the bin_means list

        # Use scipy to calculate the SEM for the bin
        sem = stats.sem(bin_counts)  # SEM
        bin_sem.append(sem)  # Store the SEM in the bin_sem list

    data = []

    for i, (bin_mean, sem) in enumerate(zip(bin_means, bin_sem), start=1):
        data.append([f"Bin {i}", f"{bin_mean:.2f} cpm", f"{sem:.2f}"])

    table = tabulate(data, headers=["Bin", "Avg (cpm)", "SEM"], tablefmt="grid")

    print(table)

    # Create a plot of counts per minute against time with error bars representing SEM
    plt.figure(figsize=(10, 6))
    plt.errorbar(
        time_bins[:-1] + bin_width_minutes / 2,  # X-axis values (middle of time bins)
        bin_means,                               # Y-axis values (mean values for each bin)
        yerr=bin_sem,                            # Error bars (SEM for each bin)
        marker='o',
        markersize=4,
        capsize=4,
        label=f'{selected_index_name} per Minute'
    )
    plt.xlabel('Time (minutes)')
    plt.ylabel(f'{selected_index_name}')

    # Check if the plot represents a Sham session from filtering criteria
    sham_detected = False  # Initialize a flag
    i = 0  # Initialize an index variable

    while i < len(filtering_criteria):
        criterion = filtering_criteria[i]

        if criterion == "Selected Header: Sham" and i + 1 < len(filtering_criteria) and filtering_criteria[i + 1] == "Filter Value: y":
            sham_detected = True
            break  # Exit the loop if Sham is detected

        i += 1

    # Set the plot title based on Sham detection
    if sham_detected:
        plot_title = f'{selected_index_name} per minute over time ({bin_width_minutes}-minute Intervals) with SEM - Sham'
    else:
        plot_title = f'{selected_index_name} per minute over time ({bin_width_minutes}-minute Intervals) with SEM'

    plt.title(plot_title)  # Set the plot title

    plt.grid(True)
    plt.legend()

    # Add a text box with filtering criteria to the far right
    text_box_content = "\n".join(filtering_criteria)
    plt.gca().text(1.02, 0.5, text_box_content, transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.8),
                    verticalalignment='center', fontsize=10)

    plt.tight_layout()  # Ensures that the text box does not interfere with other elements
    plt.show()

    # Check if the title already exists in the dictionary
    if plot_title in plotted_data:
        # If the title exists, find a unique title by appending a number
        i = 2
        while f"{plot_title} ({i})" in plotted_data:
            i += 1
        unique_plot_title = f"{plot_title} ({i})"
    else:
        unique_plot_title = plot_title
        
    # Convert NumPy arrays to lists
    x_axis_values = (time_bins[:-1] + bin_width_minutes / 2).tolist()

    # Update the plotted_data dictionary with lists instead of NumPy arrays
    plotted_data[unique_plot_title] = {
        'X-axis_values': x_axis_values,
        'Y-axis_values': bin_means,
        'SEM_values': bin_sem,
        'Label': f'{selected_index_name} per Minute',
        'X-axis_label': 'Time (minutes)',
        'Y-axis_label': selected_index_name,
        'Filtering_criteria': text_box_content,
        'Bin_data': bin_data  # You can store the bin data if needed
    }

    # Save the updated plotted_data to the JSON file
    with open("plotted_data.json", "w") as file:
        json.dump(plotted_data, file, indent=4)
    
else:
    binned_data = {}
    bin_width_minutes = float(input("Enter the bin width in minutes: "))
    for key, tuples in matching_tuples.items():
        # Extract the values for the selected index from each tuple in the list
        selected_values = [tuple[selected_index] for tuple in tuples]
        # Initialize a dictionary to store binned values for this key
        binned_values_dict = {}
        # Create time bins
        time_bins = np.arange(min_time, max_time + bin_width_minutes, bin_width_minutes)

        # Iterate through the time bins
        for bin_start, bin_end in zip(time_bins[:-1], time_bins[1:]):
            # Initialize a list to store values in this bin
            bin_values = []

            # Iterate through the data points for this key
            for start_time, end_time, value_at_selected_index in zip(start_times, end_times, selected_values):
                if start_time > bin_start and end_time <= bin_end:
                    # Value at the selected index falls within this bin for the current key
                    bin_values.append(value_at_selected_index)

            # Store the binned values for this bin in the dictionary
            binned_values_dict[(bin_start, bin_end)] = bin_values

        # Store the binned data for this key in the main dictionary
        binned_data[key] = binned_values_dict

    # Check the number of keys in the binned_data dictionary
    num_keys = len(binned_data)

    if num_keys == 1:
        # If there is only one key, calculate SEM based on the binned values for each bin
        single_key = next(iter(binned_data))  # Get the single key
        single_key_binned_values = binned_data[single_key]  # Binned values for the single key

        # Initialize a dictionary to store SEM values for each bin for the single key
        sem_per_bin = {}

        # Initialize a list to store data for the DataFrame
        data = []

        # Initialize lists to store bin information and values for plotting
        bin_starts = []
        bin_ends = []
        bin_midpoints = []
        bin_averages = []
        bin_sems = []

        # Iterate through the bins and their values
        for bin_key, bin_values in single_key_binned_values.items():
            # Calculate the SEM for this bin's values
            if len(bin_values) > 1:
                sem = np.std(bin_values, ddof=1) / np.sqrt(len(bin_values))
            else:
                sem = np.nan  # Set to NaN if there's insufficient data in the bin

            # Store the SEM value in the dictionary with the bin key as the identifier
            sem_per_bin[bin_key] = sem

            # Extract bin start and end for table
            bin_start, bin_end = bin_key
            bin_starts.append(bin_start)
            bin_ends.append(bin_end)

            # Calculate bin midpoint for plotting
            bin_midpoint = (bin_start + bin_end) / 2
            bin_midpoints.append(bin_midpoint)

            # Calculate the average for this bin
            bin_average = np.mean(bin_values)
            bin_averages.append(bin_average)

            # Store the SEM value for plotting
            bin_sems.append(sem)

            # Append the data to the list for the DataFrame
            data.append([bin_start, bin_end, bin_average, sem])  # Include SEM in the data

        # Create a DataFrame from the list
        df = pd.DataFrame(data, columns=['Bin Start', 'Bin End', 'Average', 'SEM'])

        # Print the table
        print(df)

        # Define the color for the line and error bars
        line_color = 'blue'

        # Create a line plot with error bars
        plt.figure(figsize=(10, 6))
        plt.plot(bin_midpoints, bin_averages, marker='o', color=line_color, label='Average')
        plt.errorbar(bin_midpoints, bin_averages, yerr=bin_sems, fmt='o', color=line_color, capsize=4, label='SEM')

        # Add labels and title
        plt.xlabel('Time (minutes)')
        plt.ylabel(f'Mean of {selected_index_name}')

        sham_detected = False  # Initialize a flag
        i = 0  # Initialize an index variable

        while i < len(filtering_criteria):
            criterion = filtering_criteria[i]

            if criterion == "Selected Header: Sham" and i + 1 < len(filtering_criteria) and filtering_criteria[i + 1] == "Filter Value: y":
                sham_detected = True
                break  # Exit the loop if Sham is detected

            i += 1

        # Set the plot title based on Sham detection
        if sham_detected:
            plot_title = f'{selected_index_name} Over Time ({bin_width_minutes}-minute Intervals) with SEM - Sham'
        else:
            plot_title = f'{selected_index_name} Over Time ({bin_width_minutes}-minute Intervals) with SEM'
        plt.title(plot_title)
        # Add legend
        plt.legend()

        # Add a text box with filtering criteria to the far right
        text_box_content = "\n".join(filtering_criteria)
        plt.gca().text(1.02, 0.5, text_box_content, transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.8),
                        verticalalignment='center', fontsize=10)

        # Show the plot
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()  # Ensures all elements fit within the figure
        plt.show()

        # Check if the title already exists in the dictionary
        if plot_title in plotted_data:
            # If the title exists, find a unique title by appending a number
            i = 2
            while f"{plot_title} ({i})" in plotted_data:
                i += 1
            unique_plot_title = f"{plot_title} ({i})"
        else:
            unique_plot_title = plot_title

        plotted_data[unique_plot_title] = {
            'X-axis_values': bin_midpoints,  # Use bin_midpoints as X-axis values
            'Y-axis_values': bin_averages,   # Use bin_averages as Y-axis values
            'SEM_values': bin_sems,          # Use bin_sems as SEM values
            'Label': f'{selected_index_name} over time',
            'X-axis_label': 'Time (minutes)',
            'Y-axis_label': selected_index_name,
            'Filtering_criteria': text_box_content,
            'Bin_data': bin_data,  # You can store the bin data if needed
            'Sham_detected': sham_detected  # Include Sham detection flag in the dictionary
        }

        # Save the updated plotted_data to the JSON file
        with open("plotted_data.json", "w") as file:
            json.dump(plotted_data, file, indent=4)
    else:
        # If there are multiple keys, perform the original SEM calculation across keys
        average_per_key = {}

        # Iterate through the binned data dictionary
        for key, binned_values_dict in binned_data.items():
            #print(f"Key: {key}")

            # Iterate through the bins and their values
            for bin_key, bin_values in binned_values_dict.items():
                bin_start, bin_end = bin_key
                #print(f"Bin ({bin_start:.2f} - {bin_end:.2f} minutes): {bin_values}")

            #print()  # Print an empty line to separate each key's output   

        # Initialize a dictionary to store the average of each bin for each key
        average_per_key = {}

        # Iterate through the binned data dictionary
        for key, binned_values_dict in binned_data.items():
            key_averages = {}  # Initialize a dictionary to store averages for this key

            # Iterate through the bins and their values for this key
            for bin_key, bin_values in binned_values_dict.items():
                # Check if the bin_values list is empty before calculating the mean
                if len(bin_values) > 0:
                    # Calculate the average for the current bin
                    bin_average = np.mean(bin_values)
                else:
                    bin_average = np.nan  # Set to NaN if the bin is empty

                # Store the bin average in the dictionary with the bin key as the identifier
                key_averages[bin_key] = bin_average

            # Store the average values for this key in the main dictionary
            average_per_key[key] = key_averages

            # Iterate through the average_per_key dictionary
        for key, key_averages in average_per_key.items():
            print(f"Key: {key}")

            # Iterate through the bins and their averages for this key
            for bin_key, bin_average in key_averages.items():
                bin_start, bin_end = bin_key
                print(f"Bin ({bin_start:.2f} - {bin_end:.2f} minutes) Average: {bin_average:.2f}")

            print()

        average_of_averages = {}  # Initialize a dictionary to store the average of averages for each bin
        sem_of_averages = {}      # Initialize a dictionary to store the SEM for each bin's average

        # Iterate through the bins (based on the first key, assuming all keys have the same bins)
        for bin_key in average_per_key[next(iter(average_per_key))]:
            bin_averages = []  # Initialize a list to store averages for this bin across keys

            # Iterate through the keys and their corresponding averages
            for key, key_averages in average_per_key.items():
                bin_average = key_averages.get(bin_key, np.nan)  # Get the average for this key and bin
                if not np.isnan(bin_average):
                    bin_averages.append(bin_average)

            # Calculate the average for this bin across keys and store it in the dictionary
            if bin_averages:
                bin_average_across_keys = np.mean(bin_averages)
                average_of_averages[bin_key] = bin_average_across_keys

                # Calculate the SEM for this bin's average
                sem = stats.sem(bin_averages)
                sem_of_averages[bin_key] = sem

        # Print the averages and SEMs for each bin
        for bin_key, bin_average in average_of_averages.items():
            bin_start, bin_end = bin_key
            bin_sem = sem_of_averages.get(bin_key, np.nan)
            #print(f"Bin ({bin_start:.2f} - {bin_end:.2f} minutes) Average of Averages: {bin_average:.2f} SEM: {bin_sem:.2f}")

      # Create a list to store data for the DataFrame
        data = []

        # Iterate through the bins and their averages
        for bin_key, bin_average in average_of_averages.items():
            bin_start, bin_end = bin_key
            bin_sem = sem_of_averages.get(bin_key, np.nan)

            # Append the data to the list
            data.append([bin_start, bin_end, bin_average, bin_sem])

        # Create a DataFrame from the list
        df = pd.DataFrame(data, columns=['Bin Start', 'Bin End', 'Average', 'SEM'])

        # Display the DataFrame
        print(df)

        # Extract bin information and values
        bin_starts, bin_ends = zip(*average_of_averages.keys())
        bin_averages = list(average_of_averages.values())
        bin_sems = list(sem_of_averages.values())

        # Convert bin starts and ends to midpoint values for plotting
        bin_midpoints = [(start + end) / 2 for start, end in zip(bin_starts, bin_ends)]

        # Define the color for the line and error bars
        line_color = 'blue'

        # Create a line plot with error bars
        plt.figure(figsize=(10, 6))
        plt.plot(bin_midpoints, bin_averages, marker='o', color=line_color, label='Average')
        plt.errorbar(bin_midpoints, bin_averages, yerr=bin_sems, fmt='o', color=line_color, capsize=4, label='SEM')

        # Add labels and title
        plt.xlabel('Time (minutes)')
        plt.ylabel(f'Mean of {selected_index_name}')
        sham_detected = False  # Initialize a flag
        i = 0  # Initialize an index variable

        while i < len(filtering_criteria):
            criterion = filtering_criteria[i]

            if criterion == "Selected Header: Sham" and i + 1 < len(filtering_criteria) and filtering_criteria[i + 1] == "Filter Value: y":
                sham_detected = True
                break  # Exit the loop if Sham is detected

            i += 1

        # Set the plot title based on Sham detection
        if sham_detected:
            plot_title = f'{selected_index_name} Over Time ({bin_width_minutes}-minute Intervals) with SEM - Sham'
        else:
            plot_title = f'{selected_index_name} Over Time ({bin_width_minutes}-minute Intervals) with SEM'
        plt.title(plot_title)
        # Add legend
        plt.legend()

        # Add a text box with filtering criteria to the far right
        text_box_content = "\n".join(filtering_criteria)
        plt.gca().text(1.02, 0.5, text_box_content, transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.8),
                        verticalalignment='center', fontsize=10)

        # Show the plot
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()  # Ensures all elements fit within the figure
        plt.show()

        # Check if the title already exists in the dictionary
        if plot_title in plotted_data:
            # If the title exists, find a unique title by appending a number
            i = 2
            while f"{plot_title} ({i})" in plotted_data:
                i += 1
            unique_plot_title = f"{plot_title} ({i})"
        else:
            unique_plot_title = plot_title

        plotted_data[unique_plot_title] = {
        'X-axis_values': bin_midpoints,  # Use bin_midpoints as X-axis values
        'Y-axis_values': bin_averages,   # Use bin_averages as Y-axis values
        'SEM_values': bin_sems,          # Use bin_sems as SEM values
        'Label': f'{selected_index_name} over tie',
        'X-axis_label': 'Time (minutes)',
        'Y-axis_label': selected_index_name,
        'Filtering_criteria': text_box_content,
        'Bin_data': bin_data,  # You can store the bin data if needed
        'Sham_detected': sham_detected  # Include Sham detection flag in the dictionary
    }

        # Save the updated plotted_data to the JSON file
        with open("plotted_data.json", "w") as file:
            json.dump(plotted_data, file, indent=4)

Selected Index Name: Bouts, Population Size (N): 20, Population Average: 307.0, Population Std Deviation: 176.421855509051


KeyboardInterrupt: Interrupted by user

In [306]:
# Load the plotted_data dictionary from the JSON file
try:
    with open("plotted_data.json", "r") as file:
        plotted_data = json.load(file)
except FileNotFoundError:
    plotted_data = {}  # Initialize an empty dictionary if the file doesn't exist

# List available dictionaries for plotting
available_dicts = list(plotted_data.keys())

if not available_dicts:
    print("No data available for plotting.")
    exit()

print("Available dictionaries for plotting:")
for i, dictionary in enumerate(available_dicts, 1):
    print(f"{i}. {dictionary}")

# Ask the user how they want to select dictionaries for plotting
while True:
    user_input = input("Enter 'all' to select all dictionaries, a range (e.g., '2-5'), or individual numbers (e.g., '1 3 4'): ").strip()

    if user_input.lower() == 'all':
        selected_indices = list(range(len(available_dicts)))  # Select all dictionaries
        break
    elif '-' in user_input:
        range_parts = user_input.split('-')
        if len(range_parts) == 2 and range_parts[0].isnumeric() and range_parts[1].isnumeric():
            start_idx = int(range_parts[0]) - 1
            end_idx = int(range_parts[1])
            if 0 <= start_idx < end_idx <= len(available_dicts):
                selected_indices = list(range(start_idx, end_idx))  # Select a range
                break
            else:
                print("Invalid range. Please enter a valid range.")
        else:
            print("Invalid input. Please enter a valid range (e.g., '2-5').")
    else:
        selected_numbers = user_input.split()
        if all(num.isnumeric() for num in selected_numbers):
            selected_indices = [int(num) - 1 for num in selected_numbers if 0 < int(num) <= len(available_dicts)]
            if selected_indices:
                break
            else:
                print("Invalid numbers. Please enter valid numbers.")
        else:
            print("Invalid input. Please enter 'all', a valid range, or valid numbers.")

# Initialize a dictionary to store SEM values for each time
sem_values_by_time = {}

# Initialize lists to store combined data
combined_data = []

# Initialize the plot title to None
plot_title = None

# Iterate through the selected dictionaries and combine the data
for idx in selected_indices:
    dictionary = available_dicts[idx]
    data = plotted_data[dictionary]

    x_axis_values = data.get('X-axis_values', [])
    y_axis_values = data.get('Y-axis_values', [])
    sem_values = data.get('SEM_values', [])
    label = data.get('Label', '')

    # Append the data as a tuple (x, y, sem, label) to the combined_data list
    combined_data.append((x_axis_values, y_axis_values, sem_values, label))

    # Group Y values by time
    for i, time in enumerate(x_axis_values):
        if time not in sem_values_by_time:
            sem_values_by_time[time] = []
        sem_values_by_time[time].append(y_axis_values[i])

# Create a single plot for all the combined data
for x_values, y_values, sem, label in combined_data:
    plt.errorbar(
        x_values,
        y_values,
        yerr=sem if len(sem) > 0 else None,  # Use SEM values from the dictionary if available
        marker='o',
        markersize=4,
        capsize=4,
        label=label
    )

# Set labels and title for the plot
plt.xlabel(data.get('X-axis_label', ''))
plt.ylabel(data.get('Y-axis_label', ''))

# Extract the part of the title before "SEM"
if plot_title:
    plot_title = plot_key.split("with")[0].strip()
plt.title(plot_title)  # Set the plot title

# Show grid and legend
plt.grid(True)
# Place the legend outside the plot and specify the bbox_to_anchor
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Calculate the SEM for each time and store the values
sem_values_by_time_result = []
for time, values in sem_values_by_time.items():
    if len(values) < 2:
        continue

    sem = np.std(values) / np.sqrt(len(values))
    sem_values_by_time_result.append([time, sem])  # Store time and SEM in a list

# Print the SEM values as a table with a title
table = tabulate(sem_values_by_time_result, headers=["Time", "SEM"], tablefmt="fancy_grid")
print("SEM Values of Combined Plot")
print(table)

# Display the plot
plt.show()

Available dictionaries for plotting:
1. Bouts Over Time (5.0-minute Intervals) with SEM
2. Duration Over Time (3.0-minute Intervals) with SEM
3. Duration Over Time (3.0-minute Intervals) with SEM (2)


KeyboardInterrupt: Interrupted by user

In [278]:

# Print the plotted_data dictionary
pprint.pprint(len(plotted_data))

0


In [279]:

# To print specific keys and their corresponding values, you can iterate through the dictionary
for key, value in plotted_data.items():
    print(f"Key: {key}")
    print("\n")

In [318]:
pprint.pprint(plotted_data)

{}


plotted_data.clear()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define a dictionary to map index values to names
index_name_mapping = {
    0: 'Behavior',
    1: 'Stim Count',
    2: 'Frame Rate',
    3: 'Bouts',
    4: 'Duration',
    5: 'Start Time',
    6: 'End Time'
}

def process_matching_tuples(matching_tuples):
    # Prompt the user to upload the first matching tuple and select the variable
    first_variable = input("Upload the first matching tuple. Enter a variable name for it: ")
    selected_index = int(input(f"Select an index for {first_variable}: "))
    selected_index_name = index_name_mapping[selected_index]
    selected_values = [tuple[selected_index] for tuple in matching_tuples]

    # Prompt the user to upload the second matching tuple and select the variable
    second_variable = input("Upload the second matching tuple. Enter a variable name for it: ")
    selected_index2 = int(input(f"Select an index for {second_variable}: "))
    selected_index_name2 = index_name_mapping[selected_index2]
    selected_values2 = [tuple[selected_index2] for tuple in matching_tuples]

    # Extract the start and end times from matching_tuples and convert to minutes
    start_times = [tuple[start_time_index] / 600 for tuple in matching_tuples]
    end_times = [tuple[end_time_index] / 600 for tuple in matching_tuples]

    # Define the bin width in minutes
    bin_width_minutes = 3  # Adjust the bin width as needed

    # Calculate the minimum and maximum times
    min_time = 0
    max_time = 65

    # Create time bins
    time_bins = np.arange(min_time, max_time + bin_width_minutes, bin_width_minutes)

    # Initialize lists to store counts, start times, end times, and SEM per bin for both variables
    counts1 = []
    counts2 = []
    start_times_bin = []
    end_times_bin = []
    sem_values1 = []  # To store SEM per bin for variable 1
    sem_values2 = []  # To store SEM per bin for variable 2

    # Iterate through the time bins
    for bin_start, bin_end in zip(time_bins[:-1], time_bins[1:]):
        bin_values1 = []
        bin_values2 = []

        # Iterate through the data to collect values within this bin for both variables
        for start_time, end_time, value1, value2 in zip(start_times, end_times, selected_values, selected_values2):
            if start_time > bin_start and end_time <= bin_end:
                # Data falls within this bin
                bin_values1.append(value1)
                bin_values2.append(value2)

        # Calculate the count of elements within the bin for both variables
        bin_count1 = len(bin_values1)
        bin_count2 = len(bin_values2)

        # Calculate the standard error of the mean (SEM) for both variables
        sem1 = 0  # Initialize SEM for variable 1 for this bin
        sem2 = 0  # Initialize SEM for variable 2 for this bin
        if bin_count1 > 0:
            sem1 = np.std(bin_values1, ddof=1) / np.sqrt(bin_count1) / bin_width_minutes
        if bin_count2 > 0:
            sem2 = np.std(bin_values2, ddof=1) / np.sqrt(bin_count2) / bin_width_minutes

        # Append values to respective lists
        counts1.append(bin_count1)
        counts2.append(bin_count2)
        start_times_bin.append(bin_start)
        end_times_bin.append(bin_end)
        sem_values1.append(sem1)  # Append the calculated SEM for variable 1
        sem_values2.append(sem2)  # Append the calculated SEM for variable 2

    # Calculate the difference between the bins for both variables
    difference_values = [val1 - val2 for val1, val2 in zip(counts1, counts2)]

    middle_of_bins = [start + bin_width_minutes / 2 for start in start_times_bin]

    # Create a DataFrame to store the bin statistics for both variables and the difference
    result_df = pd.DataFrame({
        'Start Time': start_times_bin,
        'End Time': end_times_bin,
        f'Sum ({first_variable})': counts1,
        f'Sum ({second_variable})': counts2,
        f'Difference ({first_variable} - {second_variable})': difference_values,
        f'SEM ({first_variable})': sem_values1,
        f'SEM ({second_variable})': sem_values2
    })

    # Display the DataFrame
    print(result_df)

    # Create a plot of the difference between counts per minute with error bars representing SEM for both variables
    plt.figure(figsize=(10, 6))
    plt.errorbar(middle_of_bins, difference_values, yerr=sem_values1, marker='o', markersize=4, capsize=4, label=f'{first_variable} - {second_variable}')
    plt.xlabel('Time (minutes)')
    plt.ylabel(f'Difference ({first_variable} - {second_variable})')
    plt.title(f'Difference between {first_variable} and {second_variable} per Minute Over Time ({bin_width_minutes}-minute Intervals) with SEM')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Assuming 'matching_tuples' contains the selected tuples
# and 'time_values' contains the corresponding time values

# Define the selected indices for start and end time
start_time_index = 5
end_time_index = 6

# Extract the name associated with the selected index
selected_index = int(input("Select an index for the variables: "))
matching_tuples = []  # Initialize the list of matching tuples

# Prompt the user to upload matching tuples and add them to the list
while True:
    user_choice = input("Do you want to upload a matching tuple? (yes/no): ")
    if user_choice.lower() == "yes":
        matching_tuple = tuple(float(x) for x in input("Enter values for the matching tuple separated by spaces: ").split())
        matching_tuples.append(matching_tuple)
    else:
        break

# Process the matching tuples and plot the difference
process_matching_tuples(matching_tuples)

In [283]:
import os

save_directory = "Z:\\KayCei\\Matching Tuples CSV"  # Replace with the desired directory path

# After filtering and obtaining matching_tuples, you can export them to a CSV file like this:
if len(matching_tuples) > 0:
    # Create a DataFrame from the list of matching tuples
    matching_tuples_df = pd.DataFrame(matching_tuples)

    # Create the full path to the CSV file in the specified directory
    csv_filename = os.path.join(save_directory, "matching_tuples.csv")

    # Export the DataFrame to the specified CSV file path
    matching_tuples_df.to_csv(csv_filename, index=False)

    print(f"Matching tuples have been exported to {csv_filename}.")
else:
    print("No matching tuples to export.")

ValueError: All arrays must be of the same length

# DONT NOT USE

In [ ]:
fps is 30  1 behavior is 1/30

In [ ]:
import ast
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def get_possible_indices(tuples):
    if tuples and isinstance(tuples[0], tuple):
        return range(len(tuples[0]))
    return []

while True:
    start_filtering = input("Do you want to start filtering? (y/n): ")
    if start_filtering.lower() != 'y':
        print("Filtering process cancelled.")
        break

    array_columns = [col for col in matching_rows.columns if "array" in col.lower()]

    print("Available columns with 'array' in their names:")
    for idx, col in enumerate(array_columns):
        print(f"{idx}: {col}")

    selected_column_index = int(input("Enter the number of the column you want to use for values: "))
    selected_column_name = array_columns[selected_column_index]

    if "array" in selected_column_name.lower():
        tuple_cells = matching_rows[selected_column_name].tolist()

        possible_indices = get_possible_indices(ast.literal_eval(tuple_cells[0]))
        print("\nAvailable indices for the elements within tuples:")
        print(possible_indices)

        selected_index = int(input("Enter the index of the element you want to plot: "))

        if selected_index == 3:
            print("Index 3 represents time, please select a different index.")
        elif selected_index==2:
            generate_duration_plot = input("Do you want to generate a duration plot? (y/n): ")
            if generate_duration_plot.lower() == 'y':
                plt.figure()

                durations = []
                max_y_length = 0
                
                for cell in tuple_cells:
                    tuple_data = ast.literal_eval(cell)
                    if len(tuple_data) > selected_index and len(tuple_data) > 4:
                        y_values = [x[selected_index] for x in tuple_data]

                        if len(y_values) > max_y_length:
                            max_y_length = len(y_values)

                        durations.append(((tuple_data[-1][3] - tuple_data[0][1]) / 10))  # Calculate duration

                x_values = list(range(1, max_y_length + 1))
                avg_durations = np.zeros(max_y_length)

                for duration in durations:
                    for i in range(len(x_values)):
                        if i < len(duration):
                            avg_durations[i] += duration[i]

                avg_durations /= len(durations)

                plt.plot(x_values, avg_durations, label='Average Duration')
                plt.xlabel('Index position')
                plt.ylabel('Average Duration (seconds)')
                plt.title(f'Average Duration Plot for Index {selected_index}')
                plt.grid(False)
                plt.legend()
                plt.show()

    continue_input = input("Do you want to continue? (y/n): ")
    if continue_input.lower() == 'n':
        break


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def bin_data(x_values, selected_index):
    # Bin the data based on x values
    bin_start = 0
    bin_increment = 300
    bins = np.arange(bin_start, max(x_values) + bin_increment, bin_increment)

    # Create a histogram
    hist, bin_edges = np.histogram(x_values, bins=bins)

    return hist, bin_edges

if x_values and y_values:
    # Call the binning function
    hist, bin_edges = bin_data(x_values, selected_index)
    print(selected_index)
    # Display the histogram
    plt.figure()
    plt.hist(x_values, bins=bin_edges, alpha=0.7)
    plt.xlabel(f'Value at Index {selected_index}')
    plt.ylabel('Frequency')
    plt.title(f'Histogram of Values at Index {selected_index}')
    plt.grid()

    plt.show()
else:
    print(f"No valid data found for plotting.")
   
# Print the values for each bin
for i, value in enumerate(hist):
    bin_center = (bin_edges[i] + bin_edges[i + 1]) / 2
    print(f"Bin {i+1}: Range {bin_edges[i]} - {bin_edges[i+1]}, Value: {bin_center}, Frequency: {value}")

plt.show()



In [ ]:
mean and sem 

In [ ]:
add index for individual tuples

In [ ]:
AD6 on a given day what is the first 

In [ ]:
## mask = []

for i, x in enumerate(tuples_list):
    found_true_for_index = False   # Reset the flag for each new row 'x'
    
    # Check if 'tup' is present in the current row 'x'
    if tup in x:
        if len(tup) > selected_index:
            # Extract the value at 'selected_index'
            value_at_selected_index = x[selected_index]
            if range_min <= value_at_selected_index <= range_max:
                mask.append(True)       # Append True to the mask
                found_true_for_index = True  # Set flag to True for this index
            else:
                mask.append(False)
        else:
            mask.append(False)
    else:
        mask.append(False)

    # Skip the rest of 'tup' once a True value is found for this index
    if found_true_for_index:
        break


In [ ]:
### Find columns with "array" in their names
array_columns = [col for col in matching_rows.columns if "array" in col.lower()]
print("Available columns with 'array' in their names:")
for idx, col in enumerate(array_columns):
    print(f"{idx}: {col}")

# Prompt user to select a column
selected_column_index = int(input("Enter the number of the column you want to use for values: "))
selected_column_name = array_columns[selected_column_index]

# Define the mapping of tuple indices to column names
tuple_index_to_column = {
    0: 'behavior',
    1: 'duration/count',
    2: 'bout',
    3: 'time (deciseconds)'
}

for idx, col_name in tuple_index_to_column.items():
    new_column_name = f"{selected_column_name}_{col_name}"
    matching_rows.loc[:, new_column_name] = None  # Add new column with None values

# Iterate through each row
for index, row in matching_rows.iterrows():
    array_data_str = row[selected_column_name]
    # Check if the content is a valid list
    try:
        array_data = ast.literal_eval(array_data_str)
        if not isinstance(array_data, list):
            continue  # Skip this row if array_data is not a list
    except (SyntaxError, ValueError, IndexError):
        print(f'Error occurred while processing array_data in row {index}: {array_data_str}')
        continue  # Skip this row if literal_eval encounters an error
    
    print(f"Extracted array_data for row {index}: {array_data}")  # Print the array_data
    
    # Process the list of tuples
    for tuple_elements in array_data:
        try:
            if not isinstance(tuple_elements, tuple):
                print("Error: This is not a tuple:", tuple_elements)
                continue
                
            processed_tuple = (float(tuple_elements[0]),) + tuple(map(int, tuple_elements[1:]))
                
            # Extract and update columns as before
            for idx, element in enumerate(processed_tuple):
                column_name = tuple_index_to_column.get(idx)
                if column_name:
                    new_column_name = f"{selected_column_name}_{column_name}"
                    if matching_rows.at[index, new_column_name] is None:
                        matching_rows.at[index, new_column_name] = []  # Initialize with empty list
                    matching_rows.at[index, new_column_name].append(element)
        except (ValueError):
            print("A value error occurred while processing tuple_elements:", tuple_elements)
        except (SyntaxError):
            print("A syntax error occurred while processing tuple_elements:", tuple_elements)

# Display the resulting DataFrame
print(matching_rows)

In [ ]:
# Create a list of newly added column names
newly_added_columns = [f"{selected_column_name}_{col_name}" for col_name in tuple_index_to_column.values()]

# Prompt user to select a column for binning
print("Available columns for binning:")
for idx, col_name in enumerate(newly_added_columns):
    print(f"{idx}: {col_name}")

selected_binning_column_index = int(input("Enter the number of the column you want to use for bins: "))

if 0 <= selected_binning_column_index < len(newly_added_columns):
    selected_binning_column_name = newly_added_columns[selected_binning_column_index]

    # Create a dictionary to store values for each row
    values_dict = {}

    # Create a dictionary to store time values for each row
    time_dictionary = {}

    for idx, row in matching_rows.iterrows():
        # Get the value of the selected column for the current row
        value = row[selected_binning_column_name]

        # Add the value to the values_dict for the current row
        values_dict[idx] = value

        # Get the time value for the current row
        time_column = [col for col in row.index if "time (deciseconds)" in col.lower()]
        if time_column:
            time_value = row[time_column[0]]
            time_dictionary[idx] = time_value

    # Display the resulting dictionaries
    print("Values Dictionary:")
    for row_index, value in values_dict.items():
        print(f"Row {row_index}: {value}")

    print("\nTime Dictionary:")
    for row_index, time_value in time_dictionary.items():
        print(f"Row {row_index}: {time_value}")
else:
    print("Invalid selection. Please choose a valid index.")



In [ ]:
# Iterate through values in the values_dict dictionary and retrieve corresponding time values
print("Values and Time Dictionary Values:")
for row_index, value in values_dict.items():
    time_value = time_dictionary.get(row_index)  # Retrieve time value using the same key
    print(f"Row {row_index}: Value: {value}, Time Value: {time_value}")



In [ ]:
3000,6000,9000,12000,15000,18000,21000,24000,27000,30000,31000,34000,39000,42000]

In [ ]:
output_path = input("Enter the full path of the directory where you want to save the CSV file: ")
output_filename = input("Enter the name of the CSV file to save the filtered data: ")
output_file_path = f"{output_path}/{output_filename}"

matching_rows.to_csv(output_file_path, index=False)

print(f"Filtered data has been saved to {output_file_path}.")

In [17]:
import ast
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def get_possible_indices(tuples):
    index_to_label = {
        0: "Behavior,0",
        1: "Stim Count,1",
        2: "Frame Rate,2",
        3: "Bouts,3",
        4: "Duration,4", 
        5: "Start Time,5", 
        6: "End Time,6"

    }
    if tuples and isinstance(tuples[0], tuple):
        return [index_to_label[i] if i in index_to_label else f"Unknown Label {i}" for i in range(len(tuples[0]))]
    return []

while True:
    start_filtering = input("Do you want to start filtering? (y/n): ")
    if start_filtering.lower() != 'y':
        print("Filtering process cancelled.")
        break

    array_columns = [col for col in matching_rows.columns if "array" in col.lower()]

    print("Available columns with 'array' in their names:")
    for idx, col in enumerate(array_columns):
        print(f"{idx}: {col}")

    selected_column_index = int(input("Enter the number of the column you want to use for values: "))
    selected_column_name = array_columns[selected_column_index]

    if "array" in selected_column_name.lower():
        tuple_cells = matching_rows[selected_column_name].tolist()

        possible_indices = get_possible_indices(tuple_cells[0])
        print("\nAvailable indices for the elements within tuples:")
        print(possible_indices)

        selected_index = int(input("Enter the index of the element you want to plot: "))
        
        if selected_index == 3:
            print("Index 3 represents time, please select a different index.")
        elif selected_index in possible_indices:
            plt.figure()

            for cell in tuple_cells:
                x_values = []
                y_values = []

                tuple_data = ast.literal_eval(cell)
                if len(tuple_data) > selected_index and len(tuple_data) > 7:
                    for x in tuple_data:
                        x_values.append(((x[3]-x[1])/10))  # Time at index 4
                        y_values.append(x[selected_index])  # Convert y value to minutes

                    if x_values and y_values:
                        title = 'Bouts' if selected_index == 2 else 'Counts'
                        plt.plot(x_values, y_values, label=f'Row {tuple_cells.index(cell) + 1}')
            
            plt.xlabel('Time (seconds)')
            plt.ylabel(f'Value at Index {selected_index} (in minutes)')
            plt.title(f'Combined Line Plot: Index {selected_index} vs. Time (Index 4) - {title}')
            plt.grid(False)
            plt.legend()
            plt.show()

    continue_input = input("Do you want to continue? (y/n): ")
    if continue_input.lower() == 'n':
        break


Do you want to start filtering? (y/n): y
Available columns with 'array' in their names:
0: Right Turn Array
1: Right Turn Array Continue
2: Locomotion Array
3: Locomotion Array Continue
4: Face Groom Array
5: Face Groom Array Continue
Enter the number of the column you want to use for values: 0

Available indices for the elements within tuples:
[]
Enter the index of the element you want to plot: 5
Do you want to continue? (y/n): n
